In [1]:
# conda install -c conda-forge pyspark 
# import findspark
# findspark.init()
from sedona.spark import SedonaContext
from sedona.register import SedonaRegistrator
from sedona.utils import SedonaKryoRegistrator, KryoSerializer
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, BooleanType, LongType, ArrayType
from pyspark.sql import Row
from pprint import pprint
from time import sleep
from pyspark.sql.window import Window
import pandas as pd
# import Transformer
import geopandas as gpd
from pyproj import Transformer
pd.set_option('display.max_columns', None)
import os
import re
from sklearn.cluster import DBSCAN
import geopandas as gpd
from pyspark.sql.functions import col, udf
!pip install clickhouse_connect
!pip install pyxlsb
!pip install osmium shapely
!pip install python-dotenv
import clickhouse_connect
from pyspark.sql.functions import split, col
from pyspark.sql.functions import sum, avg, count
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
import os
from dotenv import load_dotenv
from pyspark.sql import Window
from pyspark.sql.functions import row_number, rank, dense_rank
from pyspark.sql.functions import col, split, when, last, monotonically_increasing_id, expr 
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType, StringType
import pyspark.sql.functions as F
import builtins 
from collections import Counter


import osmnx as ox
import geopandas as gpd

load_dotenv('../.env')

CH_IP = os.getenv('CH_IP')
CLICKHOUSE_USER = os.getenv('CLICKHOUSE_USER')
CLICKHOUSE_PASSWORD = os.getenv('CLICKHOUSE_PASSWORD')

client = clickhouse_connect.get_client(host=CH_IP, port=8123, username=CLICKHOUSE_USER, password=CLICKHOUSE_PASSWORD)

packages = [
        "com.clickhouse.spark:clickhouse-spark-runtime-3.5_2.12:0.8.0"
        ,"com.clickhouse:clickhouse-jdbc:0.7.0"
        ,"com.clickhouse:clickhouse-client:0.7.0"
        ,"com.clickhouse:clickhouse-http-client:0.7.0"
        ,"org.apache.httpcomponents.client5:httpclient5:5.3.1"
        ,'org.apache.sedona:sedona-spark-3.5_2.12:1.7.0'
        ,'org.datasyslab:geotools-wrapper:1.7.0-28.5'
        ,'uk.co.gresearch.spark:spark-extension_2.12:2.11.0-3.4'
    ]

ram = 80
cpu = 4*3

spark = (
    SparkSession.builder
        .appName('appName')
        .master(f"spark://127.0.0.1:7077")
        .config("spark.jars.packages", ','.join(packages))
        .config('spark.jars.repositories', 'https://artifacts.unidata.ucar.edu/repository/unidata-all')
        .config("spark.sql.catalog.clickhouse", "com.clickhouse.spark.ClickHouseCatalog")
        .config("spark.sql.catalog.clickhouse.host", f"{CH_IP}")
        .config("spark.sql.catalog.clickhouse.protocol", "http")
        .config("spark.sql.catalog.clickhouse.http_port", "8123")
        .config("spark.sql.catalog.clickhouse.user", f"{CLICKHOUSE_USER}")
        .config("spark.sql.catalog.clickhouse.password", f"{CLICKHOUSE_PASSWORD}")
        .config("spark.sql.catalog.clickhouse.database", "default")
        .config("spark.executor.memory", f"{ram}g")
        .config("spark.driver.memory", f"{ram}g")
        .config("spark.driver.maxResultSize", f"{ram}g")
        .config("spark.executor.memoryOverhead", f"{ram}g")
        .config("spark.clickhouse.write.format", "json")
        .getOrCreate()
    )

#sedona = SedonaContext.create(spark) 
SedonaRegistrator.registerAll(spark)
spark.sql("use clickhouse")
base = '/home/sinatrafrancoise'
path_data = f"{base}/data"


https://artifacts.unidata.ucar.edu/repository/unidata-all added as a remote repository with the name: repo-1
Ivy Default Cache set to: /home/sinatrafrancoise/.ivy2/cache
The jars for the packages stored in: /home/sinatrafrancoise/.ivy2/jars
com.clickhouse.spark#clickhouse-spark-runtime-3.5_2.12 added as a dependency
com.clickhouse#clickhouse-jdbc added as a dependency
com.clickhouse#clickhouse-client added as a dependency
com.clickhouse#clickhouse-http-client added as a dependency
org.apache.httpcomponents.client5#httpclient5 added as a dependency
org.apache.sedona#sedona-spark-3.5_2.12 added as a dependency
org.datasyslab#geotools-wrapper added as a dependency
uk.co.gresearch.spark#spark-extension_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-fcdefc11-6b66-4cf4-96f7-45ca1725cbac;1.0
	confs: [default]


:: loading settings :: url = jar:file:/opt/conda/envs/spark/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found com.clickhouse.spark#clickhouse-spark-runtime-3.5_2.12;0.8.0 in central
	found com.clickhouse#clickhouse-jdbc;0.7.0 in central
	found com.clickhouse#clickhouse-client;0.7.0 in central
	found com.clickhouse#clickhouse-data;0.7.0 in central
	found com.clickhouse#clickhouse-http-client;0.7.0 in central
	found org.apache.httpcomponents.client5#httpclient5;5.3.1 in central
	found org.apache.httpcomponents.core5#httpcore5;5.2.4 in central
	found org.apache.httpcomponents.core5#httpcore5-h2;5.2.4 in central
	found org.slf4j#slf4j-api;1.7.36 in central
	found org.apache.sedona#sedona-spark-3.5_2.12;1.7.0 in central
	found org.apache.sedona#sedona-common;1.7.0 in central
	found org.apache.commons#commons-math3;3.6.1 in central
	found org.locationtech.jts#jts-core;1.20.0 in central
	found org.wololo#jts2geojson;0.16.1 in central
	found org.locationtech.spatial4j#spatial4j;0.8 in central
	found com.google.geometry#s2-geometry;2.0.0 in central
	found com.google.guava#guava;25.1-jre in centr

In [44]:
nyc_buildings = spark.read.csv(f"{path_data}/nyc_buildings.csv", header=True, inferSchema=True)
nyc_buildings.show(30, truncate=False)

+--------+-------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [3]:
#nyc_buildings.select(
    'element',
    'id',
    'geometry',
    'building',
    'height',
    'building:levels',
    'building:use',
    'building:material',
    'roof:material',
    'roof:shape',
    'roof:height',
    'addr:housenumber',
    'addr:street',
    'addr:postcode',
    'addr:city',
    'addr:state',
    'addr:country',
    'addr:district',
    'addr:suburb',
    'addr:block',
    'amenity',
    'shop',
    'office',
    'leisure',
    'healthcare',
    'tourism',
    'historic',
    'memorial',
    'religion',
    'denomination',
    'man_made',
    'industrial',
    'landuse',
    'lit',
    'access',
    'wheelchair',
    'entrance',
    'nycdoitt:bin',
    'source',
    'start_date',
    'year_of_construction',
    'building:start_date',
    'historic:civilization',
    'opening_hours',
    'usage',
    'residential',
    'building:flats',
    'surface',
    'material').write.parquet(f"{path_data}/nyc_buildings.parquet", mode='overwrite')

IndentationError: unexpected indent (1218878603.py, line 2)

In [6]:
nyc_buildings_full = spark.read.parquet(f"{path_data}/nyc_buildings.parquet")
nyc_buildings_full.show(30, truncate=False)

+-------+---------+----------------------------------------------------------------------------------------------------------------------------------------------+--------+------+---------------+------------+-----------------+-------------+----------+-----------+----------------+---------------+-------------+---------+----------+------------+-------------+-----------+----------+-------+----+------+-------+----------+-------+--------+--------+--------+------------+--------+----------+-------+----+------+----------+--------+------------+------+----------+--------------------+-------------------+---------------------+-------------+-----+-----------+--------------+-------+--------+
|element|id       |geometry                                                                                                                                      |building|height|building:levels|building:use|building:material|roof:material|roof:shape|roof:height|addr:housenumber|addr:street    |addr:postcode|addr:

In [31]:
nyc_crime = spark.read.csv(f"{path_data}/NYPD_currentyear.csv", header=True, inferSchema=True)
nyc_crime.show(5)

+----------+-----------+-------------+------------+-------------------+------------+------------+----------------+----------+-----------+-----------------+----------------+-----+-----------+-----------------+--------------------+--------+--------------------+-----+--------------------+--------------------+----------+------------+--------------+---------+--------+----------------+-------------+--------+-------+----------+----------+----------------+-----------------+--------------------+------------------------+
|CMPLNT_NUM|ADDR_PCT_CD|      BORO_NM|CMPLNT_FR_DT|       CMPLNT_FR_TM|CMPLNT_TO_DT|CMPLNT_TO_TM|CRM_ATPT_CPTD_CD|HADEVELOPT|HOUSING_PSA|JURISDICTION_CODE|      JURIS_DESC|KY_CD| LAW_CAT_CD|LOC_OF_OCCUR_DESC|           OFNS_DESC|PARKS_NM|         PATROL_BORO|PD_CD|             PD_DESC|       PREM_TYP_DESC|    RPT_DT|STATION_NAME|SUSP_AGE_GROUP|SUSP_RACE|SUSP_SEX|TRANSIT_DISTRICT|VIC_AGE_GROUP|VIC_RACE|VIC_SEX|X_COORD_CD|Y_COORD_CD|        Latitude|        Longitude|             

In [35]:
nyc_crime.createOrReplaceTempView("nyc_crime")

In [38]:
spark.sql("""
    SELECT DISTINCT year(to_date(CMPLNT_FR_DT, 'MM/dd/yyyy')) AS CMPLNT_FR_DT
    FROM nyc_crime
    ORDER BY CMPLNT_FR_DT
""").show(100)

+------------+
|CMPLNT_FR_DT|
+------------+
|        1025|
|        1919|
|        1925|
|        1970|
|        1975|
|        1979|
|        1983|
|        1984|
|        1989|
|        1991|
|        1993|
|        1994|
|        1995|
|        1997|
|        1999|
|        2000|
|        2001|
|        2002|
|        2003|
|        2004|
|        2005|
|        2006|
|        2007|
|        2008|
|        2009|
|        2010|
|        2011|
|        2012|
|        2013|
|        2014|
|        2015|
|        2016|
|        2017|
|        2018|
|        2019|
|        2020|
|        2021|
|        2022|
|        2023|
|        2024|
|        2025|
+------------+



### Loading NYC Data

In [39]:
nyc_crimes = spark.read.csv(f"{path_data}/NYPD_Arrest_Data.csv", header=True, inferSchema=True)
nyc_crimes.show(5)
nyc_crimes.createOrReplaceTempView("nyc_crimes")

+----------+-----------+-----+-----------------+-----+--------------+----------+----------+-----------+---------------+-----------------+---------+--------+---------+----------+----------+----------------+-----------------+------------------------+
|ARREST_KEY|ARREST_DATE|PD_CD|          PD_DESC|KY_CD|     OFNS_DESC|  LAW_CODE|LAW_CAT_CD|ARREST_BORO|ARREST_PRECINCT|JURISDICTION_CODE|AGE_GROUP|PERP_SEX|PERP_RACE|X_COORD_CD|Y_COORD_CD|        Latitude|        Longitude|New Georeferenced Column|
+----------+-----------+-----+-----------------+-----+--------------+----------+----------+-----------+---------------+-----------------+---------+--------+---------+----------+----------+----------------+-----------------+------------------------+
| 281369711| 01/30/2024|  177|     SEXUAL ABUSE|  116|    SEX CRIMES|PL 1306501|         F|          M|             25|                0|    25-44|       M|    BLACK|   1000558|    231080|40.8009303727402|-73.9410982410066|    POINT (-73.941098...|
| 28

In [15]:
spark.sql("""
    SELECT DISTINCT year(to_date(ARREST_DATE, 'MM/dd/yyyy')) AS ARREST_YEAR
    FROM nyc_crimes
    ORDER BY ARREST_YEAR
""").show()


+-----------+
|ARREST_YEAR|
+-----------+
|       2024|
+-----------+



In [14]:
spark.sql("""
    SELECT distinct ARREST_DATE
    FROM nyc_crimes
    ORDER BY ARREST_DATE
""").show(150)


+-----------+
|ARREST_DATE|
+-----------+
| 01/01/2024|
| 01/02/2024|
| 01/03/2024|
| 01/04/2024|
| 01/05/2024|
| 01/06/2024|
| 01/07/2024|
| 01/08/2024|
| 01/09/2024|
| 01/10/2024|
| 01/11/2024|
| 01/12/2024|
| 01/13/2024|
| 01/14/2024|
| 01/15/2024|
| 01/16/2024|
| 01/17/2024|
| 01/18/2024|
| 01/19/2024|
| 01/20/2024|
| 01/21/2024|
| 01/22/2024|
| 01/23/2024|
| 01/24/2024|
| 01/25/2024|
| 01/26/2024|
| 01/27/2024|
| 01/28/2024|
| 01/29/2024|
| 01/30/2024|
| 01/31/2024|
| 02/01/2024|
| 02/02/2024|
| 02/03/2024|
| 02/04/2024|
| 02/05/2024|
| 02/06/2024|
| 02/07/2024|
| 02/08/2024|
| 02/09/2024|
| 02/10/2024|
| 02/11/2024|
| 02/12/2024|
| 02/13/2024|
| 02/14/2024|
| 02/15/2024|
| 02/16/2024|
| 02/17/2024|
| 02/18/2024|
| 02/19/2024|
| 02/20/2024|
| 02/21/2024|
| 02/22/2024|
| 02/23/2024|
| 02/24/2024|
| 02/25/2024|
| 02/26/2024|
| 02/27/2024|
| 02/28/2024|
| 02/29/2024|
| 03/01/2024|
| 03/02/2024|
| 03/03/2024|
| 03/04/2024|
| 03/05/2024|
| 03/06/2024|
| 03/07/2024|
| 03/08/2024|
| 03/0

In [40]:
nyc_districts = (
    spark.read.format("geojson").option("multiLine", "true").load(f"{path_data}/nyc_districts.json")
    .selectExpr("explode(features) as features")
    .select("features.*")
    .withColumn("OBJECTID", expr("properties['OBJECTID']"))
    .withColumn("BoroCD", expr("properties['BoroCD']"))
    .drop("properties")
    .drop("type")
    .drop("id")
)
nyc_districts.show(truncate=True)

+--------------------+--------+------+
|            geometry|OBJECTID|BoroCD|
+--------------------+--------+------+
|POLYGON ((-73.970...|       1|   308|
|POLYGON ((-73.801...|       2|   410|
|POLYGON ((-73.881...|       3|   206|
|POLYGON ((-73.867...|       4|   480|
|POLYGON ((-73.984...|       5|   104|
|POLYGON ((-73.980...|       6|   107|
|POLYGON ((-74.193...|       7|   503|
|MULTIPOLYGON (((-...|       8|   595|
|POLYGON ((-73.877...|       9|   207|
|POLYGON ((-73.925...|      10|   317|
|POLYGON ((-73.840...|      11|   481|
|POLYGON ((-73.872...|      12|   227|
|POLYGON ((-73.820...|      13|   409|
|POLYGON ((-73.903...|      14|   316|
|POLYGON ((-73.728...|      15|   413|
|POLYGON ((-73.799...|      16|   483|
|POLYGON ((-73.895...|      17|   208|
|POLYGON ((-73.878...|      18|   226|
|POLYGON ((-73.991...|      19|   312|
|POLYGON ((-73.959...|      20|   314|
+--------------------+--------+------+
only showing top 20 rows



In [41]:
nyc_districts.createOrReplaceTempView("nyc_districts")

In [42]:
crime_nyc = spark.sql("""
    with crimes as (
                    select *
            ,ST_Point(Longitude , Latitude) as geom
        from nyc_crimes
        where Latitude is not null or Longitude is not null
    ),
    districs as (
    select 
        OBJECTID
        ,BoroCD as district
        ,geometry as geom
    from nyc_districts
    )
                      select
                        crimes.*
                      , districs.district
                        , ST_Intersects(crimes.geom, districs.geom) as intersects
                      from crimes
                      join districs on ST_Intersects(crimes.geom, districs.geom)
""")

crime_nyc.write.mode('overwrite').parquet(f"{path_data}/crime_nyc.parquet")
crime_nyc.show(5)

+----------+-----------+-----+-----------------+-----+--------------+----------+----------+-----------+---------------+-----------------+---------+--------+---------+----------+----------+----------------+-----------------+------------------------+--------------------+--------+----------+
|ARREST_KEY|ARREST_DATE|PD_CD|          PD_DESC|KY_CD|     OFNS_DESC|  LAW_CODE|LAW_CAT_CD|ARREST_BORO|ARREST_PRECINCT|JURISDICTION_CODE|AGE_GROUP|PERP_SEX|PERP_RACE|X_COORD_CD|Y_COORD_CD|        Latitude|        Longitude|New Georeferenced Column|                geom|district|intersects|
+----------+-----------+-----+-----------------+-----+--------------+----------+----------+-----------+---------------+-----------------+---------+--------+---------+----------+----------+----------------+-----------------+------------------------+--------------------+--------+----------+
| 281369711| 01/30/2024|  177|     SEXUAL ABUSE|  116|    SEX CRIMES|PL 1306501|         F|          M|             25|           

In [43]:
crime_nyc.select("District").distinct().count()

71

In [46]:
crime_nyc.createOrReplaceTempView("crime_nyc")

In [ ]:
nyc_buildings = nyc_buildings.where(F.col("geometry").like("POLYGON ((%"))

In [ ]:
nyc_buildings.createOrReplaceTempView("nyc_buildings")

In [ ]:
crime_nyc.createOrReplaceTempView("crime_nyc")

nyc = spark.sql("""
    with buildings as (
        select 
            id
            ,ST_GeomFromWKT(geometry) as geom
        from nyc_buildings
    )
    select 
        nb.id as building_id
        ,nb.geom as building_geom
        ,cn.district as district
        ,cn.geom as crime_geom
        ,ST_Distance(nb.geom, cn.geom) as distance
    from buildings nb
    join crime_nyc cn 
    on ST_Distance(nb.geom, cn.geom) = 0
""")

nyc.write.mode('overwrite').parquet(f"{path_data}/nyc_full.parquet")
nyc.show(5)


+-----------+--------------------+--------+--------------------+--------+
|building_id|       building_geom|district|          crime_geom|distance|
+-----------+--------------------+--------+--------------------+--------+
|  271585435|POLYGON ((-73.944...|     111|POINT (-73.944419...|     0.0|
|  271585435|POLYGON ((-73.944...|     111|POINT (-73.944419...|     0.0|
|  271585435|POLYGON ((-73.944...|     111|POINT (-73.944419...|     0.0|
|  265795690|POLYGON ((-73.943...|     111|POINT (-73.943488...|     0.0|
|  265795690|POLYGON ((-73.943...|     111|POINT (-73.943488...|     0.0|
+-----------+--------------------+--------+--------------------+--------+
only showing top 5 rows



In [11]:
nyc_districts = spark.read.parquet(f"{path_data}/nyc_full.parquet")

In [12]:
nyc_districts = nyc_districts.withColumnRenamed("district", "District")
nyc_districts.show(5)

+-----------+--------------------+--------+--------------------+--------+
|building_id|       building_geom|District|          crime_geom|distance|
+-----------+--------------------+--------+--------------------+--------+
|  285901149|POLYGON ((-73.922...|     205|POINT (-73.92227 ...|     0.0|
|  285901149|POLYGON ((-73.922...|     205|POINT (-73.92227 ...|     0.0|
|  297834679|POLYGON ((-73.907...|     405|POINT (-73.907688...|     0.0|
|  298198826|POLYGON ((-73.918...|     304|POINT (-73.918849...|     0.0|
|  298198832|POLYGON ((-73.935...|     303|POINT (-73.935223...|     0.0|
+-----------+--------------------+--------+--------------------+--------+
only showing top 5 rows



In [13]:
result = nyc_districts.join(crime_nyc.drop("district"), nyc_districts.crime_geom == crime_nyc.geom, "inner")
result.show(5)

+-----------+--------------------+--------+--------------------+--------+----------+-----------+-----+--------------------+-----+----------------+----------+----------+-----------+---------------+-----------------+---------+--------+--------------+----------+----------+----------------+------------------+------------------------+--------------------+----------+
|building_id|       building_geom|District|          crime_geom|distance|ARREST_KEY|ARREST_DATE|PD_CD|             PD_DESC|KY_CD|       OFNS_DESC|  LAW_CODE|LAW_CAT_CD|ARREST_BORO|ARREST_PRECINCT|JURISDICTION_CODE|AGE_GROUP|PERP_SEX|     PERP_RACE|X_COORD_CD|Y_COORD_CD|        Latitude|         Longitude|New Georeferenced Column|                geom|intersects|
+-----------+--------------------+--------+--------------------+--------+----------+-----------+-----+--------------------+-----+----------------+----------+----------+-----------+---------------+-----------------+---------+--------+--------------+----------+----------+--

In [14]:
enriched_df = result.join(nyc_buildings_full.drop("addr:district"), result.building_id == nyc_buildings_full.id, "inner")
enriched_df.show(5)

+-----------+--------------------+--------+--------------------+--------+----------+-----------+-----+--------------------+-----+--------------------+----------+----------+-----------+---------------+-----------------+---------+--------+--------------+----------+----------+-----------------+-----------------+------------------------+--------------------+----------+-------+---------+--------------------+-------------+------+---------------+------------+-----------------+-------------+----------+-----------+----------------+--------------------+-------------+---------+----------+------------+-----------+----------+-------+----+------+-------+----------+-------+--------+--------+--------+------------+--------+----------+-------+----+------+----------+--------+------------+------+----------+--------------------+-------------------+---------------------+-------------+-----+-----------+--------------+-------+--------+
|building_id|       building_geom|District|          crime_geom|distance|A

In [ ]:
enriched_df = enriched_df.withColumnRenamed("addr:district", "null_distr")

In [ ]:
enriched_df = enriched_df.drop("district")
enriched_df

DataFrame[building_id: string, ARREST_KEY: int, ARREST_DATE: string, PD_CD: int, PD_DESC: string, KY_CD: int, OFNS_DESC: string, LAW_CODE: string, LAW_CAT_CD: string, ARREST_BORO: string, ARREST_PRECINCT: int, JURISDICTION_CODE: int, AGE_GROUP: string, PERP_SEX: string, PERP_RACE: string, X_COORD_CD: int, Y_COORD_CD: int, Latitude: double, Longitude: double, New Georeferenced Column: string, geom: udt, element: string, id: string, geometry: string, building: string, height: string, building:levels: double, building:use: string, building:material: string, roof:material: string, roof:shape: string, addr:street: string, addr:postcode: string, addr:state: string, addr:suburb: string, amenity: string, man_made: string, wheelchair: string, opening_hours: string, building:flats: int]

In [ ]:
null_counts = {}

for col_name, col_type in enriched_df.dtypes:
    if col_type in ["float", "double", "int"]:
        null_condition = (F.col(col_name).isNull() | F.isnan(F.col(col_name)))
    else:
        null_condition = F.col(col_name).isNull()

    null_count = (
        enriched_df
        .withColumn("new_column", F.when(null_condition, 1).otherwise(0))
        .agg(F.sum("new_column").alias("temp_sum"))
        .collect()[0][0]
    )

    null_counts[col_name] = null_count

total_count = builtins.sum(null_counts.values())

print(null_counts)
print(total_count)
print(f"Total count of null values: {total_count}")
print(f"Total count of rows: {enriched_df.count()}")

{'building_id': 0, 'ARREST_KEY': 0, 'ARREST_DATE': 0, 'PD_CD': 11, 'PD_DESC': 0, 'KY_CD': 29, 'OFNS_DESC': 0, 'LAW_CODE': 0, 'LAW_CAT_CD': 10, 'ARREST_BORO': 0, 'ARREST_PRECINCT': 0, 'JURISDICTION_CODE': 0, 'AGE_GROUP': 0, 'PERP_SEX': 0, 'PERP_RACE': 0, 'X_COORD_CD': 0, 'Y_COORD_CD': 0, 'Latitude': 0, 'Longitude': 0, 'New Georeferenced Column': 0, 'geom': 0, 'element': 0, 'id': 0, 'geometry': 0, 'building': 0, 'height': 1227514, 'building:levels': 954374, 'building:use': 1231514, 'building:material': 1233450, 'roof:material': 1233450, 'roof:shape': 977521, 'addr:street': 975568, 'addr:postcode': 412467, 'addr:state': 407448, 'addr:suburb': 1233450, 'amenity': 1231509, 'man_made': 1233425, 'wheelchair': 671627, 'nycdoitt:bin': 1231076, 'opening_hours': 1231514, 'building:flats': 1233450}
16719407
Total count of null values: 16719407
Total count of rows: 1233450


In [17]:
columns_to_drop = ["building_geom","tourism","building:material","roof:material", "roof:height", "industrial","healthcare","nycdoitt:bin",   "denomination",  "crime_geom", "historic", "distance","entrance", "access","industrial",  "intersects", "memorial", "null_distr", "religion", "addr:housenumber", "addr:city" , "addr:country", "addr:block", "shop" ,"office","start_date", "lit",   "leisure" ,"landuse",  "year_of_construction", "residential", "usage", "historic:civilization","material", "surface",  "building:start_date" , "source"]
enriched_df = enriched_df.drop(*columns_to_drop)


In [18]:
enriched_df.show(5)

+-----------+--------+----------+-----------+-----+--------------------+-----+--------------------+----------+----------+-----------+---------------+-----------------+---------+--------+--------------+----------+----------+-----------------+------------------+------------------------+--------------------+-------+---------+--------------------+-------------+------+---------------+------------+----------+--------------------+-------------+----------+-----------+-------+--------+----------+-------------+--------------+
|building_id|District|ARREST_KEY|ARREST_DATE|PD_CD|             PD_DESC|KY_CD|           OFNS_DESC|  LAW_CODE|LAW_CAT_CD|ARREST_BORO|ARREST_PRECINCT|JURISDICTION_CODE|AGE_GROUP|PERP_SEX|     PERP_RACE|X_COORD_CD|Y_COORD_CD|         Latitude|         Longitude|New Georeferenced Column|                geom|element|       id|            geometry|     building|height|building:levels|building:use|roof:shape|         addr:street|addr:postcode|addr:state|addr:suburb|amenity|man_ma

### Loading to ClickHouse

In [ ]:
client.command("CREATE DATABASE IF NOT EXISTS crime_data")

In [ ]:
def full_func_creation_table(df, db, table):

    types = {"string": "String",
             "int": "Int32",
            "bigint": "Int64",
            "long": "Int64",
            "double": "Float64",
            "float": "Float32",
            "boolean": "Boolean",
            "timestamp": "DateTime",
            "date": "Date"
            }
    columns = [f"`{column_name}` {types.get(column_type, 'String')}" for column_name, column_type in df.dtypes]
    return f"create table if not exists {db}.{table} ({','.join(columns)}) ENGINE=MergeTree() order by {df.columns[0]}"

In [ ]:
client.command(full_func_creation_table(enriched_df, 'crime_data', 'nyc'))

In [ ]:
enriched_df.writeTo("crime_data.nyc").append()

### RGB Data and aggregations

In [50]:
df_px = spark.read.csv(f"{path_data}/rgb_districtsnyc.csv", header=True, inferSchema=True)
df_px.show()

+---+---+---+--------+
|  R|  G|  B|District|
+---+---+---+--------+
|254|255|255|     201|
|255|254|255|     201|
|253|255|254|     201|
|255|255|253|     201|
|255|255|253|     201|
|254|254|255|     201|
|254|255|255|     201|
|255|255|253|     201|
|255|254|255|     201|
|254|255|255|     201|
|255|253|254|     201|
|255|255|251|     201|
|254|255|255|     201|
|254|255|255|     201|
|254|255|253|     201|
|255|254|255|     201|
|255|254|252|     201|
|254|255|255|     201|
|254|255|255|     201|
|253|254|255|     201|
+---+---+---+--------+
only showing top 20 rows



In [51]:
df_px.select("District").distinct().count()

42

In [52]:
aggreg_df = df_px.groupby("District").agg(
    F.avg("R").alias("R_avg"),
    F.stddev("R").alias("R_stddev"),
    F.expr("percentile_approx(R, 0.5)").alias("R_median"),
    F.expr("percentile_approx(R, 0.25)").alias("R_q25"),
    F.expr("percentile_approx(R, 0.75)").alias("R_q75"),
    
    F.avg("G").alias("G_avg"),
    F.stddev("G").alias("G_stddev"),
    F.expr("percentile_approx(G, 0.5)").alias("G_median"),
    F.expr("percentile_approx(G, 0.25)").alias("G_q25"),
    F.expr("percentile_approx(G, 0.75)").alias("G_q75"),
    
    F.avg("B").alias("B_avg"),
    F.stddev("B").alias("B_stddev"),
    F.expr("percentile_approx(B, 0.5)").alias("B_median"),
    F.expr("percentile_approx(B, 0.25)").alias("B_q25"),
    F.expr("percentile_approx(B, 0.75)").alias("B_q75"),
    F.avg( (F.col("R") + F.col("G") + F.col("B")) / 3 ).alias("brightness_avg"),
    F.avg( F.when( (F.col("R")+F.col("G")+F.col("B"))/3 < 50, 1).otherwise(0) ).alias("dark_pixel_ratio"),

)
aggreg_df.show()

+--------+------------------+------------------+--------+-----+-----+------------------+------------------+--------+-----+-----+------------------+------------------+--------+-----+-----+------------------+--------------------+
|District|             R_avg|          R_stddev|R_median|R_q25|R_q75|             G_avg|          G_stddev|G_median|G_q25|G_q75|             B_avg|          B_stddev|B_median|B_q25|B_q75|    brightness_avg|    dark_pixel_ratio|
+--------+------------------+------------------+--------+-----+-----+------------------+------------------+--------+-----+-----+------------------+------------------+--------+-----+-----+------------------+--------------------+
|     211| 152.5937747819191| 56.98654506605884|     135|  107|  191|125.04837430610627| 68.84321103108333|     100|   71|  165| 78.88996827914353| 86.02875245369897|      39|   18|  109|118.84403912238967| 0.06641554321966693|
|     210|169.54456951627623|  64.3848252890404|     157|  113|  250|146.81959233343474|

In [55]:
aggreg_df.createOrReplaceTempView("aggreg_df")

nyc = spark.sql("""
        select     
            a.R_avg
            ,a.G_avg
            ,a.B_avg    
            ,a.R_stddev
            ,a.G_stddev
            ,a.B_stddev
            ,a.R_median
            ,a.G_median
            ,a.B_median
            ,a.R_q25
            ,a.G_q25
            ,a.B_q25
            ,a.R_q75
            ,a.G_q75
            ,a.B_q75
            ,a.brightness_avg
            ,a.dark_pixel_ratio            
            ,cr.*
        from aggreg_df a        
        FULL OUTER JOIN crime_nyc cr
        on a.District = cr.District
""")

nyc.show(5)


+-----+-----+-----+--------+--------+--------+--------+--------+--------+-----+-----+-----+-----+-----+-----+--------------+----------------+----------+-----------+-----+--------------------+-----+--------------------+----------+----------+-----------+---------------+-----------------+---------+--------+--------------+----------+----------+----------------+-----------------+------------------------+--------------------+--------+----------+
|R_avg|G_avg|B_avg|R_stddev|G_stddev|B_stddev|R_median|G_median|B_median|R_q25|G_q25|B_q25|R_q75|G_q75|B_q75|brightness_avg|dark_pixel_ratio|ARREST_KEY|ARREST_DATE|PD_CD|             PD_DESC|KY_CD|           OFNS_DESC|  LAW_CODE|LAW_CAT_CD|ARREST_BORO|ARREST_PRECINCT|JURISDICTION_CODE|AGE_GROUP|PERP_SEX|     PERP_RACE|X_COORD_CD|Y_COORD_CD|        Latitude|        Longitude|New Georeferenced Column|                geom|district|intersects|
+-----+-----+-----+--------+--------+--------+--------+--------+--------+-----+-----+-----+-----+-----+-----+---

In [56]:
nyc.select("District").distinct().count()

71

In [58]:
from pyspark.sql.functions import to_date, year, month, dayofmonth

df = nyc.withColumn("ARREST_DATE_DT", to_date("ARREST_DATE", "MM/dd/yyyy")) \
       .withColumn("ARREST_YEAR", year("ARREST_DATE_DT")) \
       .withColumn("ARREST_MONTH", month("ARREST_DATE_DT")) \
       .withColumn("ARREST_DAY", dayofmonth("ARREST_DATE_DT"))

df.select("ARREST_MONTH", "District").distinct().groupBy("ARREST_MONTH", "District").count().orderBy("ARREST_MONTH", "District").show()
aggdf = df.groupBy("District", "ARREST_YEAR", "ARREST_Month").agg(F.count("*").alias("total_quantity_crime_in_dist")).orderBy("District", "ARREST_YEAR", "ARREST_Month")

+------------+--------+-----+
|ARREST_MONTH|District|count|
+------------+--------+-----+
|           1|     101|    1|
|           1|     102|    1|
|           1|     103|    1|
|           1|     104|    1|
|           1|     105|    1|
|           1|     106|    1|
|           1|     107|    1|
|           1|     108|    1|
|           1|     109|    1|
|           1|     110|    1|
|           1|     111|    1|
|           1|     112|    1|
|           1|     164|    1|
|           1|     201|    1|
|           1|     202|    1|
|           1|     203|    1|
|           1|     204|    1|
|           1|     205|    1|
|           1|     206|    1|
|           1|     207|    1|
+------------+--------+-----+
only showing top 20 rows



In [59]:
df = df.join(aggdf, on=["District", "ARREST_YEAR", "ARREST_Month"], how="left")


In [ ]:
agg_sex_df = df.groupBy("District").agg(
    avg(when(df.PERP_SEX == 'M', 1).otherwise(0)).alias("mean_M"),
    avg(when(df.PERP_SEX == 'F', 1).otherwise(0)).alias("mean_F")
).orderBy("District")
df = df.join(agg_sex_df, on="District", how="left")

In [65]:
from pyspark.sql.functions import when, avg, countDistinct, count, col, row_number

agg_age_df = df.groupBy("District").agg(
    avg(when(df.AGE_GROUP == '18-24', 1/7).otherwise(0)).alias("pct_age_18_24"),
    avg(when(df.AGE_GROUP == '25-44', 1/20).otherwise(0)).alias("pct_age_25_44"),
    avg(when(df.AGE_GROUP == '45-64', 1).otherwise(0)).alias("pct_age_45_64"),
    avg(when(df.AGE_GROUP == '<18', 1).otherwise(0)).alias("pct_age_<18"),
    avg(when(df.AGE_GROUP == '65+', 1).otherwise(0)).alias("pct_age_65_plus")
)
df = df.join(agg_age_df, on="District", how="left")

In [22]:
nyc.write.mode('overwrite').parquet(f"{path_data}/nyc_full_px.parquet")

In [34]:
final_nyc = spark.read.parquet(f"{path_data}/nyc_full_px.parquet")
final_nyc.show(5)

+------------------+------------------+-----------------+-----------------+-----------------+-----------------+--------+--------+--------+-----+-----+-----+-----+-----+-----+------------------+--------------------+-----------+--------+----------+-----------+-----+--------------------+-----+--------------------+----------+----------+-----------+---------------+-----------------+---------+--------+---------+----------+----------+-----------------+------------------+------------------------+--------------------+-------+---------+--------------------+--------------+------+---------------+------------+----------+-----------+-------------+----------+-----------+-------+--------+----------+-------------+--------------+
|             R_avg|             G_avg|            B_avg|         R_stddev|         G_stddev|         B_stddev|R_median|G_median|B_median|R_q25|G_q25|B_q25|R_q75|G_q75|B_q75|    brightness_avg|    dark_pixel_ratio|building_id|District|ARREST_KEY|ARREST_DATE|PD_CD|             P

In [35]:
from pyspark.sql.functions import to_date, year, month, dayofmonth

df = final_nyc.withColumn("ARREST_DATE_DT", to_date("ARREST_DATE", "MM/dd/yyyy")) \
       .withColumn("ARREST_YEAR", year("ARREST_DATE_DT")) \
       .withColumn("ARREST_MONTH", month("ARREST_DATE_DT")) \
       .withColumn("ARREST_DAY", dayofmonth("ARREST_DATE_DT"))


In [36]:
df.select("ARREST_MONTH", "District").distinct().groupBy("ARREST_MONTH", "District").count().orderBy("ARREST_MONTH", "District").show()


+------------+--------+-----+
|ARREST_MONTH|District|count|
+------------+--------+-----+
|           1|     209|    1|
|           1|     301|    1|
|           1|     302|    1|
|           1|     303|    1|
|           1|     306|    1|
|           1|     312|    1|
|           1|     402|    1|
|           1|     404|    1|
|           2|     209|    1|
|           2|     212|    1|
|           2|     301|    1|
|           2|     302|    1|
|           2|     303|    1|
|           2|     306|    1|
|           2|     312|    1|
|           2|     313|    1|
|           2|     402|    1|
|           2|     404|    1|
|           2|     405|    1|
|           3|     209|    1|
+------------+--------+-----+
only showing top 20 rows



In [37]:
aggdf = df.groupBy("District", "ARREST_YEAR", "ARREST_Month").agg(F.count("*").alias("total_quantity_crime_in_dist")).orderBy("District", "ARREST_YEAR", "ARREST_Month")

In [38]:
df = df.join(aggdf, on=["District", "ARREST_YEAR", "ARREST_Month"], how="left")

df.show(10)

+--------+-----------+------------+------------------+------------------+-----------------+-----------------+-----------------+-----------------+--------+--------+--------+-----+-----+-----+-----+-----+-----+------------------+--------------------+-----------+----------+-----------+-----+--------------------+-----+--------------------+----------+----------+-----------+---------------+-----------------+---------+--------+---------+----------+----------+-----------------+------------------+------------------------+--------------------+-------+---------+--------------------+--------------+------+---------------+------------+----------+-----------+-------------+----------+-----------+-------+--------+----------+-------------+--------------+--------------+----------+----------------------------+
|District|ARREST_YEAR|ARREST_MONTH|             R_avg|             G_avg|            B_avg|         R_stddev|         G_stddev|         B_stddev|R_median|G_median|B_median|R_q25|G_q25|B_q25|R_q75|G_

In [39]:
agg_sex_df = df.groupBy("District").agg(
    avg(when(df.PERP_SEX == 'M', 1).otherwise(0)).alias("mean_M"),
    avg(when(df.PERP_SEX == 'F', 1).otherwise(0)).alias("mean_F")
).orderBy("District")

In [40]:
df = df.join(agg_sex_df, on="District", how="left")


In [30]:
df.show(10)

+--------+-----------+------------+------------------+------------------+-----------------+-----------------+-----------------+-----------------+--------+--------+--------+-----+-----+-----+-----+-----+-----+------------------+--------------------+-----------+----------+-----------+-----+--------------------+-----+--------------------+----------+----------+-----------+---------------+-----------------+---------+--------+---------+----------+----------+-----------------+------------------+------------------------+--------------------+-------+---------+--------------------+--------------+------+---------------+------------+----------+-----------+-------------+----------+-----------+-------+--------+----------+-------------+--------------+--------------+----------+----------------------------+------------------+-------------------+
|District|ARREST_YEAR|ARREST_MONTH|             R_avg|             G_avg|            B_avg|         R_stddev|         G_stddev|         B_stddev|R_median|G_med

In [41]:
from pyspark.sql.functions import when, avg, countDistinct, count, col, row_number

agg_age_df = df.groupBy("District").agg(
    avg(when(df.AGE_GROUP == '18-24', 1/7).otherwise(0)).alias("pct_age_18_24"),
    avg(when(df.AGE_GROUP == '25-44', 1/20).otherwise(0)).alias("pct_age_25_44"),
    avg(when(df.AGE_GROUP == '45-64', 1).otherwise(0)).alias("pct_age_45_64"),
    avg(when(df.AGE_GROUP == '<18', 1).otherwise(0)).alias("pct_age_<18"),
    avg(when(df.AGE_GROUP == '65+', 1).otherwise(0)).alias("pct_age_65_plus"),
    countDistinct("AGE_GROUP").alias("num_age_groups_present")
)

agg_age_df.show(100)

+--------+--------------------+--------------------+-------------------+--------------------+--------------------+----------------------+
|District|       pct_age_18_24|       pct_age_25_44|      pct_age_45_64|         pct_age_<18|     pct_age_65_plus|num_age_groups_present|
+--------+--------------------+--------------------+-------------------+--------------------+--------------------+----------------------+
|     212|0.027649769585253448| 0.03064516129032256|0.08602150537634409|0.021505376344086023| 0.08602150537634409|                     5|
|     402| 0.03353423954860234| 0.03361759425493622|0.09290843806104129|                 0.0|                 0.0|                     3|
|     209| 0.04220779220779226| 0.02386363636363611|0.22727272727272727|                 0.0|                 0.0|                     3|
|     405|0.023809523809523805| 0.03333333333333335|0.16666666666666666|                 0.0|                 0.0|                     3|
|     304|                 0.0|   

In [ ]:
age_counts = df.groupBy("District", "AGE_GROUP").count()

windowSpec = Window.partitionBy("District").orderBy(col("count").desc())

age_counts = age_counts.withColumn("rank", row_number().over(windowSpec))
dominant_age_df = age_counts.filter(col("rank") == 1).select(
    "District", 
    col("AGE_GROUP").alias("dominant_age_group_in_district")
)

dominant_age_df.show(100)

+--------+------------------------------+
|District|dominant_age_group_in_district|
+--------+------------------------------+
|     205|                         25-44|
|     209|                         25-44|
|     212|                         25-44|
|     301|                         25-44|
|     302|                         25-44|
|     303|                         25-44|
|     304|                         25-44|
|     305|                         25-44|
|     306|                         25-44|
|     311|                         25-44|
|     312|                         25-44|
|     313|                         25-44|
|     314|                         25-44|
|     315|                         25-44|
|     402|                         25-44|
|     403|                         25-44|
|     404|                         25-44|
|     405|                         25-44|
|     409|                         25-44|
+--------+------------------------------+



In [ ]:
df = df.join(agg_age_df, on="District", how="left")


In [34]:
df.show(10)

+--------+-----------+------------+------------------+------------------+-----------------+-----------------+-----------------+-----------------+--------+--------+--------+-----+-----+-----+-----+-----+-----+------------------+--------------------+-----------+----------+-----------+-----+--------------------+-----+--------------------+----------+----------+-----------+---------------+-----------------+---------+--------+---------+----------+----------+-----------------+------------------+------------------------+--------------------+-------+---------+--------------------+--------------+------+---------------+------------+----------+-----------+-------------+----------+-----------+-------+--------+----------+-------------+--------------+--------------+----------+----------------------------+------------------+-------------------+-------------------+-------------------+-------------------+-----------+---------------+----------------------+
|District|ARREST_YEAR|ARREST_MONTH|             

In [43]:
df.write.mode('overwrite').parquet(f"{path_data}/nyc_full_px_enriched.parquet")

25/06/12 20:41:39 WARN StandaloneAppClient$ClientEndpoint: Connection to 86f0235d7cff:7077 failed; waiting for master to reconnect...
25/06/12 20:41:39 WARN StandaloneSchedulerBackend: Disconnected from Spark cluster! Waiting for reconnection...
25/06/12 20:41:44 ERROR TaskSchedulerImpl: Lost executor 0 on 172.18.0.9: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.


In [2]:
nyc_file = spark.read.parquet(f"{path_data}/nyc_full_px_enriched.parquet")
nyc_file.show(10)

25/06/14 20:28:36 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------+-----------+------------+------------------+------------------+-----------------+-----------------+-----------------+-----------------+--------+--------+--------+-----+-----+-----+-----+-----+-----+------------------+--------------------+-----------+----------+-----------+-----+--------------------+-----+--------------------+----------+----------+-----------+---------------+-----------------+---------+--------+---------+----------+----------+-----------------+------------------+------------------------+--------------------+-------+---------+--------------------+--------------+------+---------------+------------+----------+-----------+-------------+----------+-----------+-------+--------+----------+-------------+--------------+--------------+----------+----------------------------+------------------+-------------------+-------------------+-------------------+-------------------+-----------+---------------+----------------------+
|District|ARREST_YEAR|ARREST_MONTH|             

In [ ]:
nyc_file.select("District").distinct().count()

19

In [ ]:
nyc_file.select("OFNS_DESC").distinct().orderBy("OFNS_DESC").show(100)

+--------------------+
|           OFNS_DESC|
+--------------------+
| ADMINISTRATIVE CODE|
|ANTICIPATORY OFFE...|
|ASSAULT 3 & RELAT...|
|     BURGLAR'S TOOLS|
|            BURGLARY|
|CRIMINAL MISCHIEF...|
|   CRIMINAL TRESPASS|
|     DANGEROUS DRUGS|
|   DANGEROUS WEAPONS|
|  DISORDERLY CONDUCT|
|      FELONY ASSAULT|
|             FORGERY|
|              FRAUDS|
|            GAMBLING|
|       GRAND LARCENY|
|GRAND LARCENY OF ...|
|INTOXICATED & IMP...|
|            JOSTLING|
|KIDNAPPING & RELA...|
|MISCELLANEOUS PEN...|
|OFF. AGNST PUB OR...|
|OFFENSES AGAINST ...|
|OFFENSES AGAINST ...|
|OFFENSES INVOLVIN...|
|OTHER OFFENSES RE...|
|    OTHER STATE LAWS|
|OTHER STATE LAWS ...|
|OTHER TRAFFIC INF...|
|       PETIT LARCENY|
|POSSESSION OF STO...|
|             ROBBERY|
|VEHICLE AND TRAFF...|
+--------------------+



In [24]:

top_offenses = (
    nyc_file.groupBy("OFNS_DESC")
    .count()
    .orderBy("count", ascending=False)
    .limit(15)
    .select("OFNS_DESC")
    .rdd.flatMap(lambda x: x)
    .collect()
)


In [26]:
from pyspark.sql.functions import when, avg, count

agg_exprs = [
    avg(when(col("OFNS_DESC") == offense, 1).otherwise(0)).alias(f"pct_{offense.lower().replace(' ', '_')}")
    for offense in top_offenses
]

agg_offense_df = nyc_file.groupBy("District", "ARREST_YEAR", "ARREST_MONTH").agg(*agg_exprs)

agg_offense_df.show(50)


+--------+-----------+------------+--------------------+-----------------------------------+--------------------------------+--------------------+--------------------+--------------------+---------------------+----------------------------------+--------------------+------------+---------------------+-------------------+--------------------+---------------------------+----------------------------------+
|District|ARREST_YEAR|ARREST_MONTH|   pct_petit_larceny|pct_other_offenses_related_to_theft|pct_assault_3_&_related_offenses|pct_other_state_laws| pct_dangerous_drugs|   pct_grand_larceny|pct_dangerous_weapons|pct_offenses_against_public_admini|         pct_forgery|pct_burglary|pct_criminal_trespass| pct_felony_assault|          pct_frauds|pct_miscellaneous_penal_law|pct_criminal_mischief_&_related_of|
+--------+-----------+------------+--------------------+-----------------------------------+--------------------------------+--------------------+--------------------+--------------------+

In [ ]:

offense_counts = nyc_file.groupBy("District", "ARREST_YEAR", "ARREST_MONTH", "OFNS_DESC") \
    .agg(count("*").alias("crime_count"))


In [7]:
offense_counts.show(10)

+--------+-----------+------------+--------------------+-----------+
|District|ARREST_YEAR|ARREST_MONTH|           OFNS_DESC|crime_count|
+--------+-----------+------------+--------------------+-----------+
|     404|       2024|           8|            BURGLARY|          4|
|     403|       2024|           3|CRIMINAL MISCHIEF...|          6|
|     402|       2024|          12|      FELONY ASSAULT|          7|
|     404|       2024|           6|    OTHER STATE LAWS|        164|
|     306|       2024|           8|OFFENSES INVOLVIN...|        109|
|     306|       2024|           9|OFFENSES AGAINST ...|        109|
|     402|       2024|           6|       PETIT LARCENY|        290|
|     301|       2024|           5|OTHER OFFENSES RE...|          6|
|     306|       2024|           5|OTHER OFFENSES RE...|        109|
|     402|       2024|           4|POSSESSION OF STO...|          7|
+--------+-----------+------------+--------------------+-----------+
only showing top 10 rows



In [10]:
total_counts = nyc_file.groupBy("District", "ARREST_YEAR", "ARREST_MONTH") \
    .agg(count("*").alias("total_crimes"))
total_counts.show(10)

+--------+-----------+------------+------------+
|District|ARREST_YEAR|ARREST_MONTH|total_crimes|
+--------+-----------+------------+------------+
|     402|       2024|           7|         421|
|     403|       2024|           3|           9|
|     306|       2024|           2|         655|
|     212|       2024|           4|          13|
|     212|       2024|           2|           8|
|     306|       2024|           3|         981|
|     404|       2024|           3|        5820|
|     404|       2024|           9|        2055|
|     402|       2024|           9|           4|
|     209|       2024|          10|         176|
+--------+-----------+------------+------------+
only showing top 10 rows



In [11]:
offense_with_total = offense_counts.join(
    total_counts,
    on=["District", "ARREST_YEAR", "ARREST_MONTH"],
    how="left"
)
offense_with_total.show(10)

+--------+-----------+------------+--------------------+-----------+------------+
|District|ARREST_YEAR|ARREST_MONTH|           OFNS_DESC|crime_count|total_crimes|
+--------+-----------+------------+--------------------+-----------+------------+
|     404|       2024|           8|            BURGLARY|          4|        4672|
|     403|       2024|           3|CRIMINAL MISCHIEF...|          6|           9|
|     402|       2024|          12|      FELONY ASSAULT|          7|          21|
|     404|       2024|           6|    OTHER STATE LAWS|        164|        7253|
|     306|       2024|           8|OFFENSES INVOLVIN...|        109|         654|
|     306|       2024|           9|OFFENSES AGAINST ...|        109|        1090|
|     402|       2024|           6|       PETIT LARCENY|        290|         290|
|     301|       2024|           5|OTHER OFFENSES RE...|          6|         111|
|     306|       2024|           5|OTHER OFFENSES RE...|        109|         981|
|     402|      

In [ ]:
nyc_file.select("ARREST_MONTH", "District").distinct().count()

115

In [22]:
nyc_file.createOrReplaceTempView("nyc_file")

n = spark.sql("""
    select
        count(*), 
        District 
    from nyc_file
               group by District
""")
n.show(100)

+--------+--------+
|count(1)|District|
+--------+--------+
|      93|     212|
|    2228|     402|
|    1936|     209|
|      36|     405|
|       1|     304|
|   52788|     404|
|       9|     403|
|     534|     301|
|   11891|     306|
|      10|     205|
|   14400|     303|
|       1|     305|
|   98301|     302|
|       1|     314|
|      25|     313|
|       1|     315|
|     144|     312|
|       1|     409|
|       1|     311|
+--------+--------+



In [5]:
ny_build = spark.read.csv(f"{path_data}/buildings.csv", header=True, inferSchema=True)
ny_build.show(5)

+--------------------+----+-------+--------+----------+--------+-----------------+------------+---------------+----------------+-----------+--------------------+----------------+-------------+
|            the_geom|NAME|    BIN|DOITT_ID|  BASE_BBL|OBJECTID|Construction Year|Feature Code|Geometry Source|Ground Elevation|Height Roof|    LAST_EDITED_DATE|LAST_STATUS_TYPE|Map Pluto BBL|
+--------------------+----+-------+--------+----------+--------+-----------------+------------+---------------+----------------+-----------+--------------------+----------------+-------------+
|POINT (-73.946696...|NULL|3032417|  672968|3012560076| 1026585|             1910|        2100|Photogrammetric|              97|      25.87|01/28/2020 05:16:...|     Constructed|   3012560076|
|POINT (-73.821842...|NULL|4140548|  328604|4064170022|  807856|             1940|        2100|Photogrammetric|              46|      27.03|08/22/2017 07:31:...|     Constructed|   4064170022|
|POINT (-73.815321...|NULL|4481285|

In [6]:
ny_build.select(
    "the_geom",
    "Construction Year",
    "Geometry Source",
    "Ground Elevation",
    "Height Roof",
    "LAST_STATUS_TYPE"
).write.parquet(f"{path_data}/ny_buildings.parquet", mode='overwrite')

In [7]:
ny_build = spark.read.parquet(f"{path_data}/ny_buildings.parquet")

### MODEL TRAINING


In [3]:
nyc_nyc = spark.read.parquet(f"{path_data}/nyc_full_px.parquet")
nyc_nyc.show(5)

25/06/09 10:23:34 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------------------+------------------+------------------+-----------+--------+----------+-----------+-----+--------------------+-----+--------------------+----------+----------+-----------+---------------+-----------------+---------+--------+---------+----------+----------+-----------------+------------------+------------------------+--------------------+--------+--------+--------------------+-------------+------+---------------+------------+-----------------+-------------+----------+-----------+-------------+----------+-----------+-------+--------+----------+-------------+--------------+
|             R_avg|             G_avg|             B_avg|building_id|district|ARREST_KEY|ARREST_DATE|PD_CD|             PD_DESC|KY_CD|           OFNS_DESC|  LAW_CODE|LAW_CAT_CD|ARREST_BORO|ARREST_PRECINCT|JURISDICTION_CODE|AGE_GROUP|PERP_SEX|PERP_RACE|X_COORD_CD|Y_COORD_CD|         Latitude|         Longitude|New Georeferenced Column|                geom| element|      id|            geometry|     b

In [8]:
ny_build.createOrReplaceTempView("buildings_types")
nyc_nyc.createOrReplaceTempView("districts")

build = spark.sql("""
WITH buildings_prepared AS (
    SELECT 
        *,
        ST_GeomFromWKT(the_geom) AS geom_point
    FROM buildings_types
),

districts_prepared AS (
    SELECT 
        district,
        ST_GeomFromWKT(geometry) AS geom_polygon
    FROM districts
)

SELECT   
    b.*,
    d.district,
    ST_Intersects(d.geom_polygon, b.geom_point) AS intersects
FROM buildings_prepared b
JOIN districts_prepared d
    ON ST_Intersects(d.geom_polygon, b.geom_point)
""")


In [9]:
build.write.mode('overwrite').parquet(f"{path_data}/buildings_data.parquet")

In [17]:
buildings = spark.read.parquet(f"{path_data}/buildings_data.parquet")
buildings.show(100)

+--------------------+-----------------+---------------+----------------+-----------+----------------+--------------------+--------+----------+
|            the_geom|Construction Year|Geometry Source|Ground Elevation|Height Roof|LAST_STATUS_TYPE|          geom_point|district|intersects|
+--------------------+-----------------+---------------+----------------+-----------+----------------+--------------------+--------+----------+
|POINT (-73.976912...|             2003| Other (Manual)|              51|       57.0|     Constructed|POINT (-73.976912...|     306|      true|
|POINT (-73.976912...|             2003| Other (Manual)|              51|       57.0|     Constructed|POINT (-73.976912...|     306|      true|
|POINT (-73.976912...|             2003| Other (Manual)|              51|       57.0|     Constructed|POINT (-73.976912...|     306|      true|
|POINT (-73.976912...|             2003| Other (Manual)|              51|       57.0|     Constructed|POINT (-73.976912...|     306|    

In [18]:
res = buildings.select("district").distinct().count()
res

9

In [41]:
agg_dist = buildings.groupby("district").agg(
    F.count("*").alias("total_buildings_in_district"),
    F.avg("Height Roof").alias("avg_height_roof"),
    F.avg("Ground Elevation").alias("avg_ground_elevation"),
    F.avg("Construction Year").alias("avg_construction_year"),
    F.expr("percentile_approx(`Construction Year`, 0.5)").alias("median_construction_year"),
    F.expr("percentile_approx(`Ground Elevation`, 0.25)").alias("q25_ground_elevation"),
    F.expr("percentile_approx(`Ground Elevation`, 0.75)").alias("q75_ground_elevation"),
    F.expr("percentile_approx(`Height Roof`, 0.5)").alias("median_height_roof"),
    F.expr("percentile_approx(`Height Roof`, 0.25)").alias("q25_height_roof"),
    F.expr("percentile_approx(`Height Roof`, 0.75)").alias("q75_height_roof"),
    
).orderBy("district")

In [42]:
agg_dist.createOrReplaceTempView("buildings")
df.createOrReplaceTempView("df")

full = spark.sql("""
        select 
                 b.total_buildings_in_district,
                 b.avg_height_roof,
                 b.avg_ground_elevation,
                 b.avg_construction_year,
                 b.median_construction_year,
                 b.q25_ground_elevation,
                 b.q75_ground_elevation,
                 b.median_height_roof,
                 b.q25_height_roof,
                 b.q75_height_roof,
                 d.*
                 from buildings b
                 join df d
                 on b.district = d.district

""")

In [ ]:
full.write.mode('overwrite').parquet(f"{path_data}/nyc_full_buildings.parquet")

In [18]:
df = spark.read.parquet(f"{path_data}/nyc_full_buildings.parquet")
df.show(105)

+--------+-----------+------------+----------------------------+-------------+-------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------+------------+------------+---------+---------+---------+---------+---------+---------+-------------------------------+------------------+--------------------+------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+
|District|ARREST_YEAR|ARREST_Month|total_quantity_crime_in_dist|sum_OFNS_DESC|sum_PERP_RACE|             avg_R|             avg_G|             avg_B|      avg_R_stddev|      avg_G_stddev|     avg_B_stddev|avg_R_median|avg_G_median|avg_B_median|avg_R_q25|avg_G_q25|avg_B_q25|avg_R_q75|avg_G_q75|avg_B_q75|avg_total_buildings_in_district|    avg_brightness|avg_dark_pixel_ratio|        avg_mean_M|          avg_mean_F|   avg_pct_age_18_24|   avg_pct_age_25_44|  avg_pct_age_45_64| 

In [17]:
df = df.groupby("District", "ARREST_YEAR", "ARREST_Month").agg(
    F.count("*").alias("total_quantity_crime_in_dist"),
    F.sum("OFNS_DESC").alias("sum_OFNS_DESC"),
    F.sum("PERP_RACE").alias("sum_PERP_RACE"),
    F.avg("R_avg").alias("avg_R"),
    F.avg("G_avg").alias("avg_G"),
    F.avg("B_avg").alias("avg_B"),
    F.avg("R_stddev").alias("avg_R_stddev"),
    F.avg("G_stddev").alias("avg_G_stddev"),
    F.avg("B_stddev").alias("avg_B_stddev"),
    F.avg("R_median").alias("avg_R_median"),
    F.avg("G_median").alias("avg_G_median"),
    F.avg("B_median").alias("avg_B_median"),
    F.avg("R_q25").alias("avg_R_q25"),
    F.avg("G_q25").alias("avg_G_q25"),
    F.avg("B_q25").alias("avg_B_q25"),
    F.avg("R_q75").alias("avg_R_q75"),
    F.avg("G_q75").alias("avg_G_q75"),
    F.avg("B_q75").alias("avg_B_q75"),
    F.avg("total_buildings_in_district").alias("avg_total_buildings_in_district"),
    F.avg("brightness_avg").alias("avg_brightness"),
    F.avg("dark_pixel_ratio").alias("avg_dark_pixel_ratio"),
    F.avg("mean_M").alias("avg_mean_M"),
    F.avg("mean_F").alias("avg_mean_F"),
    F.avg("pct_age_18_24").alias("avg_pct_age_18_24"),
    F.avg("pct_age_25_44").alias("avg_pct_age_25_44"),
    F.avg("pct_age_45_64").alias("avg_pct_age_45_64"),
    F.avg("pct_age_<18").alias("avg_pct_age_<18"),
    F.avg("pct_age_65_plus").alias("avg_pct_age_65_plus")
)





df.write.mode('overwrite').parquet(f"{path_data}/nyc_full_buildings.parquet")

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `OFNS_DESC` cannot be resolved. Did you mean one of the following? [`sum_OFNS_DESC`, `avg_B`, `avg_G`, `avg_R`, `ARREST_YEAR`].;
'Aggregate [District#1567L, ARREST_YEAR#1568, ARREST_Month#1569], [District#1567L, ARREST_YEAR#1568, ARREST_Month#1569, count(1) AS total_quantity_crime_in_dist#1787L, sum('OFNS_DESC) AS sum_OFNS_DESC#1789, sum('PERP_RACE) AS sum_PERP_RACE#1791, avg('R_avg) AS avg_R#1793, avg('G_avg) AS avg_G#1795, avg('B_avg) AS avg_B#1797, avg('R_stddev) AS avg_R_stddev#1799, avg('G_stddev) AS avg_G_stddev#1801, avg('B_stddev) AS avg_B_stddev#1803, avg('R_median) AS avg_R_median#1805, avg('G_median) AS avg_G_median#1807, avg('B_median) AS avg_B_median#1809, avg('R_q25) AS avg_R_q25#1811, avg('G_q25) AS avg_G_q25#1813, avg('B_q25) AS avg_B_q25#1815, avg('R_q75) AS avg_R_q75#1817, avg('G_q75) AS avg_G_q75#1819, avg('B_q75) AS avg_B_q75#1821, avg('total_buildings_in_district) AS avg

In [ ]:
df.createOrReplaceTempView("df")

n = spark.sql("""
    select
              ARREST_Month,
        count(*), 
        District 
    from df
            group by District , ARREST_Month
            Order by district , ARREST_Month , count(*)
""")
n.show(100)


+------------+--------+--------+
|ARREST_Month|count(1)|District|
+------------+--------+--------+
|           6|       6|     205|
|           7|       1|     205|
|          12|       3|     205|
|           1|      88|     209|
|           2|      88|     209|
|           3|     220|     209|
|           4|     264|     209|
|           5|     440|     209|
|           7|     220|     209|
|           8|     176|     209|
|           9|     132|     209|
|          10|     176|     209|
|          11|      88|     209|
|          12|      44|     209|
|           2|       8|     212|
|           4|      13|     212|
|           7|      23|     212|
|           8|       7|     212|
|           9|      16|     212|
|          10|      24|     212|
|          12|       2|     212|
|           1|      67|     301|
|           2|      42|     301|
|           3|      12|     301|
|           4|      32|     301|
|           5|     111|     301|
|           6|      52|     301|
|         

In [37]:
buildings = buildings.join(agg_dist, on="district", how="left")
buildings.show(100)

+--------+--------------------+-----------------+---------------+----------------+-----------+----------------+--------------------+----------+---------------------------+----------------+--------------------+---------------------+------------------------+--------------------+--------------------+------------------+---------------+---------------+
|district|            the_geom|Construction Year|Geometry Source|Ground Elevation|Height Roof|LAST_STATUS_TYPE|          geom_point|intersects|total_buildings_in_district| avg_height_roof|avg_ground_elevation|avg_construction_year|median_construction_year|q25_ground_elevation|q75_ground_elevation|median_height_roof|q25_height_roof|q75_height_roof|
+--------+--------------------+-----------------+---------------+----------------+-----------+----------------+--------------------+----------+---------------------------+----------------+--------------------+---------------------+------------------------+--------------------+--------------------+--

In [11]:
buildings = buildings.withColumnRenamed("Construction Year", "construction_year") \
                        .withColumnRenamed("the_geom", "geometry_from_build") \
                     .withColumnRenamed("LAST_STATUS_TYPE", "last_status_type")
buildings =  buildings.drop("geom_point","intersects", "district")


In [ ]:
buildings.show(100)

+--------------------+-----------------+---------------+----------------+-----------+----------------+
| geometry_from_build|construction_year|Geometry Source|Ground Elevation|Height Roof|last_status_type|
+--------------------+-----------------+---------------+----------------+-----------+----------------+
|POINT (-73.978008...|             1920|Photogrammetric|              47|      20.73|     Constructed|
|POINT (-73.978008...|             1920|Photogrammetric|              47|      20.73|     Constructed|
|POINT (-73.978008...|             1920|Photogrammetric|              47|      20.73|     Constructed|
|POINT (-73.978008...|             1920|Photogrammetric|              47|      20.73|     Constructed|
|POINT (-73.978008...|             1920|Photogrammetric|              47|      20.73|     Constructed|
|POINT (-73.978008...|             1920|Photogrammetric|              47|      20.73|     Constructed|
|POINT (-73.978008...|             1920|Photogrammetric|              47|

In [14]:
full = full.drop("geometry_from_build", "building_id" "geom_point", "New Georeferenced Column", "geom_polygon", "intersects" ,"building:material", "roof:material" )
full.show(10)

+-----------------+---------------+----------------+-----------+----------------+--------------------+------------------+------------------+------------------+-----------+--------+----------+-----------+-----+--------------------+-----+--------------------+----------+----------+-----------+---------------+-----------------+---------+--------+---------+----------+----------+-----------------+------------------+--------------------+--------+--------+--------------------+-------------+------+---------------+------------+----------+-----------+-------------+----------+-----------+-------+--------+----------+-------------+--------------+
|construction_year|Geometry Source|Ground Elevation|Height Roof|last_status_type|          geom_point|             R_avg|             G_avg|             B_avg|building_id|district|ARREST_KEY|ARREST_DATE|PD_CD|             PD_DESC|KY_CD|           OFNS_DESC|  LAW_CODE|LAW_CAT_CD|ARREST_BORO|ARREST_PRECINCT|JURISDICTION_CODE|AGE_GROUP|PERP_SEX|PERP_RACE|X_COO

In [15]:
desired_first_columns = [
    "id", "district", "geometry", "geom", "building", 
    "construction_year", "Ground Elevation", "Height Roof", "last_status_type", 
    "ARREST_KEY", "ARREST_DATE"
]

remaining_columns = [col for col in full.columns if col not in desired_first_columns]
new_columns_order = desired_first_columns + remaining_columns
reord_full = full.select(new_columns_order)
reord_full.show(5)


+--------+--------+--------------------+--------------------+-------------+-----------------+----------------+-----------+----------------+----------+-----------+---------------+--------------------+------------------+------------------+------------------+-----------+-----+--------------------+-----+--------------------+----------+----------+-----------+---------------+-----------------+---------+--------+---------+----------+----------+-----------------+------------------+--------+------+---------------+------------+----------+-----------+-------------+----------+-----------+-------+--------+----------+-------------+--------------+
|      id|district|            geometry|                geom|     building|construction_year|Ground Elevation|Height Roof|last_status_type|ARREST_KEY|ARREST_DATE|Geometry Source|          geom_point|             R_avg|             G_avg|             B_avg|building_id|PD_CD|             PD_DESC|KY_CD|           OFNS_DESC|  LAW_CODE|LAW_CAT_CD|ARREST_BORO|ARR

In [ ]:
chicago_crimes = spark.read.csv(f"{path_data}/Crimes_2001_to_Present.csv", header=True, inferSchema=True)
chicago_crimes.show(5)

+--------+-----------+--------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|                Date|               Block|IUCR|        Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+--------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|13311263|   JG503434|07/29/2022 03:39:...|     023XX S TROY ST|1582|OFFENSE INVOLVING...|   CHILD PORNOGRAPHY|           RE

In [ ]:
chicago_dist = spark.read.csv(f"{path_data}/ChicagoDistricts.csv", header=True, inferSchema=True)
chicago_dist.show(5)

+--------------------+----------+--------+
|            the_geom|DIST_LABEL|DIST_NUM|
+--------------------+----------+--------+
|MULTIPOLYGON (((-...|      17TH|      17|
|MULTIPOLYGON (((-...|      20TH|      20|
|MULTIPOLYGON (((-...|      31ST|      31|
|MULTIPOLYGON (((-...|      31ST|      31|
|MULTIPOLYGON (((-...|      19TH|      19|
+--------------------+----------+--------+
only showing top 5 rows



In [ ]:
chicago_dist.createOrReplaceTempView("chicago_dist")
chicago_crimes.createOrReplaceTempView("chicago_crimes")

Chicago_intersects = spark.sql("""
          With crimes as (
              select
                  *
                  ,ST_Point(Longitude, Latitude) as geom
              FROM chicago_crimes
            where Latitude is not null or Longitude is not null
          ),
          districts as (
              select
                  DIST_NUM 
                 ,DIST_LABEL
                ,ST_GeomFromWKT(the_geom) as geom
              from chicago_dist
            )
            select
              crimes.*
              ,districts.DIST_LABEL
              ,districts.DIST_NUM
              ,ST_Intersects(crimes.geom, districts.geom) as intersects
          from crimes
          join districts on ST_Intersects(crimes.geom, districts.geom)
""")


Chicago_intersects.write.mode('overwrite').parquet(f"{path_data}/Chicago_intersects.parquet")
Chicago_intersects.show(5)

25/04/27 13:28:16 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------+-----------+--------------------+--------------------+----+---------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+--------------------+----------+--------+----------+
|      ID|Case Number|                Date|               Block|IUCR|   Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|                geom|DIST_LABEL|DIST_NUM|intersects|
+--------+-----------+--------------------+--------------------+----+---------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+--------------------+----------+----

In [ ]:
spark.read.parquet(f"{path_data}/Chicago_intersects.parquet").show(5)

+-------+-----------+--------------------+--------------------+----+---------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+--------------------+----------+--------+----------+
|     ID|Case Number|                Date|               Block|IUCR|   Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|                geom|DIST_LABEL|DIST_NUM|intersects|
+-------+-----------+--------------------+--------------------+----+---------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+--------------------+----------+-------

In [ ]:
nyc_districts.toPandas().to_csv(f"{path_data}/nyc_districts.csv", index=False)


In [ ]:
crime_nyc = (
    crime_nyc.withColumnRenamed("ARREST_KEY", "ID")
        .withColumnRenamed("ARREST_DATE", "Date")
        .withColumnRenamed("X_COORD_CD", "X Coordinate")
        .withColumnRenamed("Y_COORD_CD", "Y Coordinate")
        .withColumnRenamed("PERP_RACE", "race")
        .withColumnRenamed("AGE_GROUP", "age")
        .withColumnRenamed("PERP_SEX", "sex")
        .withColumnRenamed("OFNS_DESC", "Description")
        .withColumnRenamed("district", "District")
)
crime_nyc.show(5)


+---------+----------+-----+--------------------+-----+--------------------+----------+----------+-----------+---------------+-----------------+-----+---+--------------+------------+------------+------------------+------------------+--------------------+--------------------+--------+----------+
|       ID|      Date|PD_CD|             PD_DESC|KY_CD|         Description|  LAW_CODE|LAW_CAT_CD|ARREST_BORO|ARREST_PRECINCT|JURISDICTION_CODE|  age|sex|          race|X Coordinate|Y Coordinate|          Latitude|         Longitude|             Lon_Lat|                geom|District|intersects|
+---------+----------+-----+--------------------+-----+--------------------+----------+----------+-----------+---------------+-----------------+-----+---+--------------+------------+------------+------------------+------------------+--------------------+--------------------+--------+----------+
|205590989|11/24/2019|  339|LARCENY,PETIT FRO...|  341|       PETIT LARCENY|PL 1552500|         M|          M|  

In [ ]:
crime_nyc = (
    crime_nyc
    .withColumn("Case Number", F.lit(None))
    .withColumn("Block", F.lit(None))
    .withColumn("IUCR", F.lit(None))
    .withColumn("Primary Type", F.lit(None))
    .withColumn("Location Description", F.lit(None))
    .withColumn("Arrest", F.lit(None))
    .withColumn("Domestic", F.lit(None))
    .withColumn("Beat", F.lit(None))
    .withColumn("Ward", F.lit(None))
    .withColumn("Community Area", F.lit(None))
    .withColumn("FBI Code", F.lit(None))
    .withColumn("Year", F.lit(None))
    .withColumn("Updated On", F.lit(None))
    .withColumn("Location", F.lit(None))
    .withColumn("DIST_LABEL", F.lit(None))
    .withColumn("DIST_NUM", F.lit(None))
)
Chicago_intersects = (
    Chicago_intersects
    .withColumn("PD_CD", F.lit(None))
    .withColumn("PD_DESC", F.lit(None))
    .withColumn("KY_CD", F.lit(None))
    .withColumn("LAW_CODE", F.lit(None))
    .withColumn("LAW_CAT_CD", F.lit(None))
    .withColumn("ARREST_BORO", F.lit(None))
    .withColumn("ARREST_PRECINCT", F.lit(None))
    .withColumn("JURISDICTION_CODE", F.lit(None))
    .withColumn("age", F.lit(None))
    .withColumn("sex", F.lit(None))
    .withColumn("race", F.lit(None))
    .withColumn("Lon_Lat", F.lit(None))
)

In [ ]:
merged_df = crime_nyc.unionByName(Chicago_intersects)
merged_df.show(5)

+---------+----------+-----+--------------------+-----+--------------------+----------+----------+-----------+---------------+-----------------+-----+---+--------------+------------+------------+------------------+------------------+--------------------+--------------------+--------+----------+-----------+-----+----+------------+--------------------+------+--------+----+----+--------------+--------+----+----------+--------+----------+--------+
|       ID|      Date|PD_CD|             PD_DESC|KY_CD|         Description|  LAW_CODE|LAW_CAT_CD|ARREST_BORO|ARREST_PRECINCT|JURISDICTION_CODE|  age|sex|          race|X Coordinate|Y Coordinate|          Latitude|         Longitude|             Lon_Lat|                geom|District|intersects|Case Number|Block|IUCR|Primary Type|Location Description|Arrest|Domestic|Beat|Ward|Community Area|FBI Code|Year|Updated On|Location|DIST_LABEL|DIST_NUM|
+---------+----------+-----+--------------------+-----+--------------------+----------+----------+------

In [ ]:
def full_func_creation_table(df, db, table):

    types = {"string": "String",
             "int": "Int32",
            "bigint": "Int64",
            "long": "Int64",
            "double": "Float64",
            "float": "Float32",
            "boolean": "Boolean",
            "timestamp": "DateTime",
            "date": "Date"
            }
    columns = [f"`{column_name}` {types.get(column_type, 'String')}" for column_name, column_type in df.dtypes]
    return f"create table if not exists {db}.{table} ({','.join(columns)}) ENGINE=MergeTree() order by {df.columns[0]}"


In [ ]:
client.command(full_func_creation_table(merged_df, 'crime_data', 'crime_merge'))

In [ ]:
merged_df.writeTo("crime_data.crime_merge").append()

25/04/21 13:00:07 WARN StandaloneAppClient$ClientEndpoint: Connection to 56bff44e4ea3:7077 failed; waiting for master to reconnect...
25/04/21 13:00:07 WARN StandaloneSchedulerBackend: Disconnected from Spark cluster! Waiting for reconnection...
25/04/21 13:00:12 ERROR TaskSchedulerImpl: Lost executor 0 on 172.18.0.5: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.


In [ ]:

crime_nyc = (
    crime_nyc.withColumn("Year", F.lit(None))
             .withColumn("FBI Code", F.lit(None))
             .withColumn("Community Area", F.lit(None))
             .withColumn("Location", F.lit(None))
             .withColumn("Primary Type", F.lit(None))
             .withColumn("Location Description", F.lit(None))
             .withColumn("IUCR", F.lit(None))
             .withColumn("Ward", F.lit(None))
             .withColumn("Updated On", F.lit(None))
             .withColumn("Block", F.lit(None))
             .withColumn("DIST_NUM", F.lit(None))
             .withColumn("Domestic", F.lit(None))
             .withColumn("District", F.lit(None))
             .withColumn("Case Number", F.lit(None))
             .withColumn("Arrest", F.lit(None))
)


crime_nyc.show(5)

+---------+----------+-----+--------------------+-----+--------------------+----------+----------+-----------+---------------+-----------------+-----+---+--------------+------------+------------+------------------+------------------+--------------------+--------------------+--------+----------+----+--------+--------------+--------+------------+--------------------+----+----+----------+-----+--------+--------+-----------+------+
|       ID|      Date|PD_CD|             PD_DESC|KY_CD|         Description|  LAW_CODE|LAW_CAT_CD|ARREST_BORO|ARREST_PRECINCT|JURISDICTION_CODE|  age|sex|          race|X Coordinate|Y Coordinate|          Latitude|         Longitude|             Lon_Lat|                geom|District|intersects|Year|FBI Code|Community Area|Location|Primary Type|Location Description|IUCR|Ward|Updated On|Block|DIST_NUM|Domestic|Case Number|Arrest|
+---------+----------+-----+--------------------+-----+--------------------+----------+----------+-----------+---------------+----------

In [ ]:
Chicago_intersects = (
    Chicago_intersects.withColumn("ARREST_BORO", F.lit(None))
                      .withColumn("sex", F.lit(None))
                      .withColumn("KY_CD", F.lit(None))
                      .withColumn("PD_DESC", F.lit(None))
                      .withColumn("age", F.lit(None))
                      .withColumn("LAW_CAT_CD", F.lit(None))
                      .withColumn("JURISDICTION_CODE", F.lit(None))
                      .withColumn("district", F.lit(None))
                      .withColumn("ARREST_PRECINCT", F.lit(None))
                      .withColumn("PD_CD", F.lit(None))
                      .withColumn("Lon_Lat", F.lit(None))
                      .withColumn("LAW_CODE", F.lit(None))
                      .withColumn("race", F.lit(None))
)


Chicago_intersects.show(5)

+--------+-----------+--------------------+--------------------+----+---------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+--------------------+--------+----------+-----------+----+-----+-------+----+----------+-----------------+---------------+-----+-------+--------+----+
|      ID|Case Number|                Date|               Block|IUCR|   Primary Type|         Description|Location Description|Arrest|Domestic|Beat|district|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|                geom|DIST_NUM|intersects|ARREST_BORO| sex|KY_CD|PD_DESC| age|LAW_CAT_CD|JURISDICTION_CODE|ARREST_PRECINCT|PD_CD|Lon_Lat|LAW_CODE|race|
+--------+-----------+--------------------+--------------------+----+---------------+--------------------+--------------------

In [ ]:
crime_nyc.unionByName(Chicago_intersects).show(5)

+---------+----------+-----+--------------------+-----+--------------------+----------+----------+-----------+---------------+-----------------+-----+---+--------------+------------+------------+------------------+------------------+--------------------+--------------------+--------+----------+----+--------+--------------+--------+------------+--------------------+----+----+----------+-----+--------+--------+-----------+------+----+
|       id|      date|pd_cd|             pd_desc|ky_cd|         description|  law_code|law_cat_cd|arrest_boro|arrest_precinct|jurisdiction_code|  age|sex|          race|x coordinate|y coordinate|          latitude|         longitude|             lon_lat|                geom|district|intersects|year|fbi code|community area|location|primary type|location description|iucr|ward|updated on|block|dist_num|domestic|case number|arrest|beat|
+---------+----------+-----+--------------------+-----+--------------------+----------+----------+-----------+---------------+

In [ ]:
# Add Chicago-only columns to NYC
crime_nyc = (
    crime_nyc.withColumn("year", F.lit(None))
             .withColumn("fbi code", F.lit(None))
             .withColumn("community area", F.lit(None))
             .withColumn("location", F.lit(None))
             .withColumn("primary type", F.lit(None))
             .withColumn("location description", F.lit(None))
             .withColumn("iucr", F.lit(None))
             .withColumn("ward", F.lit(None))
             .withColumn("updated on", F.lit(None))
             .withColumn("block", F.lit(None))
             .withColumn("dist_num", F.lit(None))
             .withColumn("domestic", F.lit(None))
             .withColumn("district", F.lit(None)) 
             .withColumn("case number", F.lit(None))
             .withColumn("arrest", F.lit(None))
             .withColumn("beat", F.lit(None))
)


# Add NYC-only columns to Chicago
Chicago_intersects = (
    Chicago_intersects.withColumn("arrest_boro", F.lit(None))
                      .withColumn("sex", F.lit(None))
                      .withColumn("ky_cd", F.lit(None))
                      .withColumn("pd_desc", F.lit(None))
                      .withColumn("age", F.lit(None))
                      .withColumn("law_cat_cd", F.lit(None))
                      .withColumn("jurisdiction_code", F.lit(None))
                      .withColumn("arrest_precinct", F.lit(None))
                      .withColumn("pd_cd", F.lit(None))
                      .withColumn("lon_lat", F.lit(None))
                      .withColumn("law_code", F.lit(None))
                      .withColumn("race", F.lit(None))
)


In [ ]:
columns_chicago = set(Chicago_intersects.columns)
columns_nyc = set(crime_nyc.columns)

missing_in_nyc = columns_chicago - columns_nyc
missing_in_chicago = columns_nyc - columns_chicago
print("Missing in NYC:")
print(missing_in_nyc)

print("Missing in Chicago:")
print(missing_in_chicago)


Missing in NYC:
{'District', 'law_cat_cd', 'Updated On', 'jurisdiction_code', 'IUCR', 'ky_cd', 'DIST_NUM', 'Block', 'pd_desc', 'lon_lat', 'Primary Type', 'Location Description', 'Case Number', 'Location', 'Community Area', 'Year', 'FBI Code', 'Beat', 'law_code', 'Domestic', 'Ward', 'arrest_precinct', 'Arrest', 'pd_cd', 'arrest_boro'}
Missing in Chicago:
{'beat', 'location', 'location description', 'district', 'year', 'KY_CD', 'domestic', 'LAW_CAT_CD', 'ARREST_PRECINCT', 'ward', 'LAW_CODE', 'JURISDICTION_CODE', 'updated on', 'block', 'case number', 'ARREST_BORO', 'arrest', 'iucr', 'primary type', 'fbi code', 'community area', 'PD_CD', 'PD_DESC', 'dist_num', 'Lon_Lat'}


In [ ]:
crime_nyc = crime_nyc.select([F.col(c).alias(c.lower()) for c in crime_nyc.columns])
Chicago_intersects = Chicago_intersects.select([F.col(c).alias(c.lower()) for c in Chicago_intersects.columns])


In [ ]:
combined_df = crime_nyc.unionByName(Chicago_intersects)
combined_df.show(5)

25/04/17 19:28:17 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---------+----------+-----+--------------------+-----+--------------------+----------+----------+-----------+---------------+-----------------+-----+---+--------------+------------+------------+------------------+------------------+--------------------+--------------------+--------+----------+----+--------+--------------+--------+------------+--------------------+----+----+----------+-----+--------+--------+-----------+------+----+
|       id|      date|pd_cd|             pd_desc|ky_cd|         description|  law_code|law_cat_cd|arrest_boro|arrest_precinct|jurisdiction_code|  age|sex|          race|x coordinate|y coordinate|          latitude|         longitude|             lon_lat|                geom|district|intersects|year|fbi code|community area|location|primary type|location description|iucr|ward|updated on|block|dist_num|domestic|case number|arrest|beat|
+---------+----------+-----+--------------------+-----+--------------------+----------+----------+-----------+---------------+

In [ ]:
null_counts = {}

for col_name, col_type in combined_df.dtypes:
    if col_type in ["float", "double", "int"]:
        null_condition = (F.col(col_name).isNull() | F.isnan(F.col(col_name)))
    else:
        null_condition = F.col(col_name).isNull()

    null_count = (
        combined_df
        .withColumn("new_column", F.when(null_condition, 1).otherwise(0))
        .agg(F.sum("new_column").alias("temp_sum"))
        .collect()[0][0]
    )

    null_counts[col_name] = null_count

total_count = builtins.sum(null_counts.values())

print(null_counts)
print(total_count)
print(f"Total count of null values: {total_count}")
print(f"Total count of rows: {combined_df.count()}")


{'age': 8177185, 'arrest': 5717716, 'arrest_boro': 8177176, 'arrest_precinct': 8177168, 'beat': 5717716, 'block': 5717716, 'case number': 5717716, 'community area': 6321514, 'date': 0, 'description': 9163, 'dist_num': 5717716, 'district': 13894884, 'domestic': 5717716, 'fbi code': 5717716, 'geom': 0, 'id': 0, 'intersects': 0, 'iucr': 5717716, 'jurisdiction_code': 8177178, 'ky_cd': 8186918, 'latitude': 0, 'law_cat_cd': 8200758, 'law_code': 8177364, 'location': 5717716, 'location description': 5726836, 'lon_lat': 8177168, 'longitude': 0, 'pd_cd': 8178044, 'pd_desc': 8186331, 'primary type': 5717716, 'race': 8177168, 'sex': 8177168, 'updated on': 5717716, 'ward': 6322800, 'x coordinate': 0, 'y coordinate': 0, 'year': 5717716}
199057415
Total count of null values: 199057415


Total count of rows: 13894884


In [ ]:
def full_func_creation_table(df, db, table):

    types = {"string": "String",
             "int": "Int32",
            "bigint": "Int64",
            "long": "Int64",
            "double": "Float64",
            "float": "Float32",
            "boolean": "Boolean",
            "timestamp": "DateTime",
            "date": "Date"
            }
    columns = [f"`{column_name}` {types.get(column_type, 'String')}" for column_name, column_type in df.dtypes]
    return f"create table if not exists {db}.{table} ({','.join(columns)}) ENGINE=MergeTree() order by {df.columns[0]}"


In [ ]:
full_func_creation_table(combined_df, 'crime_data', 'crime_combined')

'create table if not exists crime_data.crime_combined (`id` Int32,`date` String,`pd_cd` Int32,`pd_desc` String,`ky_cd` Int32,`description` String,`law_code` String,`law_cat_cd` String,`arrest_boro` String,`arrest_precinct` Int32,`jurisdiction_code` Int32,`age` String,`sex` String,`race` String,`x coordinate` Float64,`y coordinate` Float64,`latitude` Float64,`longitude` Float64,`lon_lat` String,`geom` String,`district` Int32,`intersects` Boolean,`year` Int32,`fbi code` String,`community area` Int32,`location` String,`primary type` String,`location description` String,`iucr` String,`ward` Int32,`updated on` String,`block` String,`dist_num` Int32,`domestic` Boolean,`case number` String,`arrest` Boolean,`beat` Int32) ENGINE=MergeTree() order by id'

In [ ]:
client.command(full_func_creation_table(combined_df, 'crime_data', 'crime_combined'))

In [ ]:
combined_df.writeTo("crime_data.crime_combined").append()

In [ ]:
client.insert('crime_data.crime_combined', combined_df)

In [ ]:
df = combined_df.toPandas()

In [ ]:
df

,id,date,pd_cd,pd_desc,ky_cd,description,law_code,law_cat_cd,arrest_boro,arrest_precinct,jurisdiction_code,age,sex,race,x coordinate,y coordinate,latitude,longitude,lon_lat,geom,district,intersects,year,fbi code,community area,location,primary type,location description,iucr,ward,updated on,block,dist_num,domestic,case number,arrest,beat
0,205590989,11/24/2019,339.0,"LARCENY,PETIT FROM OPEN AREAS,",341.0,PETIT LARCENY,PL 1552500,M,M,28.0,0.0,25-44,F,WHITE,997571.0,234556.0,40.810476,-73.951880,POINT (-73.95188015099995 40.81047604300005),POINT (-73.95188015099995 40.81047604300005),NaN,True,NaN,None,NaN,None,None,None,None,NaN,None,None,NaN,None,None,None,NaN
1,202546976,09/17/2019,793.0,WEAPONS POSSESSION 3,118.0,DANGEROUS WEAPONS,PL 2650201,F,M,6.0,0.0,45-64,M,BLACK,982746.0,206647.0,40.733883,-74.005428,POINT (-74.00542831099995 40.73388303300004),POINT (-74.00542831099995 40.73388303300004),NaN,True,NaN,None,NaN,None,None,None,None,NaN,None,None,NaN,None,None,None,NaN
2,203563701,10/10/2019,494.0,"STOLEN PROPERTY 2,1,POSSESSION",111.0,POSSESSION OF STOLEN PROPERTY,PL 1654502,F,K,90.0,0.0,18-24,M,BLACK HISPANIC,1000830.0,197889.0,40.709829,-73.940198,POINT (-73.94019763899996 40.70982902900005),POINT (-73.94019763899996 40.70982902900005),NaN,True,NaN,None,NaN,None,None,None,None,NaN,None,None,NaN,None,None,None,NaN
3,205169609,11/14/2019,397.0,"ROBBERY,OPEN AREA UNCLASSIFIED",105.0,ROBBERY,PL 1601001,F,B,43.0,0.0,18-24,M,BLACK HISPANIC,1020183.0,239283.0,40.823387,-73.870170,POINT (-73.87017044999999 40.823387291000074),POINT (-73.87017044999999 40.823387291000074),NaN,True,NaN,None,NaN,None,None,None,None,NaN,None,None,NaN,None,None,None,NaN
4,204097687,10/23/2019,105.0,STRANGULATION 1ST,106.0,FELONY ASSAULT,PL 1211300,F,Q,114.0,0.0,25-44,M,BLACK,1007694.0,219657.0,40.769561,-73.915363,POINT (-73.91536344699995 40.769561242000066),POINT (-73.91536344699995 40.769561242000066),NaN,True,NaN,None,NaN,None,None,None,None,NaN,None,None,NaN,None,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13894879,13790609,03/29/2025 06:00:00 PM,NaN,None,NaN,ATTEMPT - AUTOMOBILE,None,None,None,NaN,NaN,None,None,None,1201525.0,1834449.0,41.700477,-87.537701,None,POINT (-87.537701103 41.700476705),4.0,True,2025.0,07,52.0,"(41.700476705, -87.537701103)",MOTOR VEHICLE THEFT,STREET,0920,10.0,04/06/2025 03:41:20 PM,107XX S AVENUE M,4.0,False,JJ199817,False,432.0
13894880,13790302,03/29/2025 07:45:00 PM,NaN,None,NaN,RETAIL THEFT,None,None,None,NaN,NaN,None,None,None,1168469.0,1929861.0,41.963071,-87.655984,None,POINT (-87.655984213 41.963070794),19.0,True,2025.0,06,3.0,"(41.963070794, -87.655984213)",THEFT,DEPARTMENT STORE,0860,46.0,04/06/2025 03:41:20 PM,044XX N BROADWAY,19.0,False,JJ199433,True,1913.0
13894881,13795591,02/20/2025 12:00:00 PM,NaN,None,NaN,OVER $500,None,None,None,NaN,NaN,None,None,None,1177518.0,1890999.0,41.856231,-87.623896,None,POINT (-87.623896406 41.856230779),1.0,True,2025.0,06,33.0,"(41.856230779, -87.623896406)",THEFT,APARTMENT,0810,3.0,04/06/2025 03:40:31 PM,019XX S MICHIGAN AVE,1.0,False,JJ205757,False,132.0
13894882,13790020,03/29/2025 03:00:00 PM,NaN,None,NaN,TELEPHONE THREAT,None,None,None,NaN,NaN,None,None,None,1161061.0,1894662.0,41.866640,-87.684200,None,POINT (-87.684200232 41.866639654),12.0,True,2025.0,08A,28.0,"(41.866639654, -87.684200232)",OTHER OFFENSE,STREET,2820,28.0,04/06/2025 03:41:20 PM,023XX W ROOSEVELT RD,12.0,False,JJ199122,False,1225.0


In [ ]:
df["latitude"].isnull()


0           False
1           False
2           False
3           False
4           False
            ...  
13894879    False
13894880    False
13894881    False
13894882    False
13894883    False
Name: latitude, Length: 13894884, dtype: bool

In [ ]:
crime_nyc = spark.read.parquet(f"{path_data}/crime_nyc.parquet")
crime_nyc.show(5)

+----------+-----------+-----+--------------------+-----+--------------------+----------+----------+-----------+---------------+-----------------+---------+--------+---------+----------+----------+------------------+------------------+--------------------+--------------------+--------+----------+
|ARREST_KEY|ARREST_DATE|PD_CD|             PD_DESC|KY_CD|           OFNS_DESC|  LAW_CODE|LAW_CAT_CD|ARREST_BORO|ARREST_PRECINCT|JURISDICTION_CODE|AGE_GROUP|PERP_SEX|PERP_RACE|X_COORD_CD|Y_COORD_CD|          Latitude|         Longitude|             Lon_Lat|                geom|district|intersects|
+----------+-----------+-----+--------------------+-----+--------------------+----------+----------+-----------+---------------+-----------------+---------+--------+---------+----------+----------+------------------+------------------+--------------------+--------------------+--------+----------+
|  26633324| 02/01/2007|  478|THEFT OF SERVICES...|  343|OTHER OFFENSES RE...|PL 1651503|         M|      

In [ ]:
chicago_df = pd.read_parquet(f"{path_data}/Chicago_intersects.parquet")
nyc_df = pd.read_parquet(f"{path_data}/crime_nyc.parquet")

In [ ]:
nyc_df["city"] = "NYC"
chicago_df["city"] = "Chicago"


In [ ]:
#chicago_df["district"] = chicago_df["Community Area"]
chicago_df = chicago_df.drop(columns=["Community Area", "DIST_NUM"])
chicago_df

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location,geom,intersects,city
0,13203321,JG415333,09/06/2023 05:00:00 PM,002XX N Wells st,1320,CRIMINAL DAMAGE,TO VEHICLE,PARKING LOT / GARAGE (NON RESIDENTIAL),False,False,122,1.0,42.0,14,1174694,1901831,2023,11/04/2023 03:40:18 PM,41.886018,-87.633938,"(41.886018055, -87.633937881)",b'\x12\x00\x00\x00\x01\x00\x00\x00\xcb\xa50p\x...,True,Chicago
1,13204489,JG416325,09/06/2023 11:00:00 AM,0000X E 8TH ST,0810,THEFT,OVER $500,PARKING LOT / GARAGE (NON RESIDENTIAL),False,False,123,1.0,4.0,06,1176857,1896680,2023,11/04/2023 03:40:18 PM,41.871835,-87.626151,"(41.871834768, -87.62615082)",b'\x12\x00\x00\x00\x01\x00\x00\x00\xdf\x90\xe3...,True,Chicago
2,11695116,JC272771,05/21/2019 08:20:00 AM,018XX S CALIFORNIA AVE,0620,BURGLARY,UNLAWFUL ENTRY,RESIDENCE,False,False,1023,10.0,25.0,05,1157982,1890961,2019,01/19/2024 03:40:50 PM,41.856547,-87.695605,"(41.856547057, -87.695604526)",b'\x12\x00\x00\x00\x01\x00\x00\x00\xa7\x87\xd8...,True,Chicago
3,12419690,JE295655,07/07/2021 10:30:00 AM,132XX S GREENWOOD AVE,1544,SEX OFFENSE,SEXUAL EXPLOITATION OF A CHILD,RESIDENCE,False,False,533,5.0,10.0,17,1186051,1817781,2021,01/19/2024 03:40:50 PM,41.655116,-87.594883,"(41.65511579, -87.594883198)",b'\x12\x00\x00\x00\x01\x00\x00\x00e\x16\xfa\x9...,True,Chicago
4,12729745,JF279458,06/14/2022 02:47:00 PM,035XX N CENTRAL AVE,0340,ROBBERY,ATTEMPT STRONG ARM - NO WEAPON,BANK,True,False,1633,16.0,30.0,03,1138391,1923142,2022,01/19/2024 03:40:50 PM,41.945232,-87.766735,"(41.945232412, -87.766734668)",b'\x12\x00\x00\x00\x01\x00\x00\x00>\xf1H.\x12\...,True,Chicago
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8177163,13790609,JJ199817,03/29/2025 06:00:00 PM,107XX S AVENUE M,0920,MOTOR VEHICLE THEFT,ATTEMPT - AUTOMOBILE,STREET,False,False,432,4.0,10.0,07,1201525,1834449,2025,04/06/2025 03:41:20 PM,41.700477,-87.537701,"(41.700476705, -87.537701103)",b'\x12\x00\x00\x00\x01\x00\x00\x00\x1f\x1a\xe3...,True,Chicago
8177164,13790302,JJ199433,03/29/2025 07:45:00 PM,044XX N BROADWAY,0860,THEFT,RETAIL THEFT,DEPARTMENT STORE,True,False,1913,19.0,46.0,06,1168469,1929861,2025,04/06/2025 03:41:20 PM,41.963071,-87.655984,"(41.963070794, -87.655984213)",b'\x12\x00\x00\x00\x01\x00\x00\x00\xbfa5\xa5\x...,True,Chicago
8177165,13795591,JJ205757,02/20/2025 12:00:00 PM,019XX S MICHIGAN AVE,0810,THEFT,OVER $500,APARTMENT,False,False,132,1.0,3.0,06,1177518,1890999,2025,04/06/2025 03:40:31 PM,41.856231,-87.623896,"(41.856230779, -87.623896406)",b'\x12\x00\x00\x00\x01\x00\x00\x00*\xf70\xeb\x...,True,Chicago
8177166,13790020,JJ199122,03/29/2025 03:00:00 PM,023XX W ROOSEVELT RD,2820,OTHER OFFENSE,TELEPHONE THREAT,STREET,False,False,1225,12.0,28.0,08A,1161061,1894662,2025,04/06/2025 03:41:20 PM,41.866640,-87.684200,"(41.866639654, -87.684200232)",b'\x12\x00\x00\x00\x01\x00\x00\x00\xc2\x16\xc5...,True,Chicago


In [ ]:
nyc_df = nyc_df.rename(columns={
    "ARREST_KEY": "ID",
    "ARREST_DATE": "Date",
    "X_COORD_CD": "X Coordinate",
    "Y_COORD_CD": "Y Coordinate",
    "PERP_RACE": "race",
    "PERP_SEX": "sex",
    "AGE_GROUP": "age",
    "JURISDICTION_CODE": "jurisdiction",
    "LAW_CODE": "law_code",
    "OFNS_DESC": "description"
})
nyc_df


,ID,Date,PD_CD,PD_DESC,KY_CD,description,law_code,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,jurisdiction,age,sex,race,X Coordinate,Y Coordinate,Latitude,Longitude,Lon_Lat,geom,district,intersects,city
0,205590989,11/24/2019,339.0,"LARCENY,PETIT FROM OPEN AREAS,",341.0,PETIT LARCENY,PL 1552500,M,M,28,0.0,25-44,F,WHITE,997571.0,234556.0,40.810476,-73.951880,POINT (-73.95188015099995 40.81047604300005),b'\x12\x00\x00\x00\x01\x00\x00\x00h\x90\xb9\x9...,110,True,NYC
1,202546976,09/17/2019,793.0,WEAPONS POSSESSION 3,118.0,DANGEROUS WEAPONS,PL 2650201,F,M,6,0.0,45-64,M,BLACK,982746.0,206647.0,40.733883,-74.005428,POINT (-74.00542831099995 40.73388303300004),b'\x12\x00\x00\x00\x01\x00\x00\x00\xe8\x8d\xfc...,102,True,NYC
2,203563701,10/10/2019,494.0,"STOLEN PROPERTY 2,1,POSSESSION",111.0,POSSESSION OF STOLEN PROPERTY,PL 1654502,F,K,90,0.0,18-24,M,BLACK HISPANIC,1000830.0,197889.0,40.709829,-73.940198,POINT (-73.94019763899996 40.70982902900005),b'\x12\x00\x00\x00\x01\x00\x00\x00\x00\xd2\xb7...,301,True,NYC
3,205169609,11/14/2019,397.0,"ROBBERY,OPEN AREA UNCLASSIFIED",105.0,ROBBERY,PL 1601001,F,B,43,0.0,18-24,M,BLACK HISPANIC,1020183.0,239283.0,40.823387,-73.870170,POINT (-73.87017044999999 40.823387291000074),"b'\x12\x00\x00\x00\x01\x00\x00\x00\x84,f\xdf\x...",209,True,NYC
4,204097687,10/23/2019,105.0,STRANGULATION 1ST,106.0,FELONY ASSAULT,PL 1211300,F,Q,114,0.0,25-44,M,BLACK,1007694.0,219657.0,40.769561,-73.915363,POINT (-73.91536344699995 40.769561242000066),b'\x12\x00\x00\x00\x01\x00\x00\x00d4\x91P\x95z...,401,True,NYC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5717711,201765665,08/30/2019,905.0,"INTOXICATED DRIVING,ALCOHOL",347.0,INTOXICATED & IMPAIRED DRIVING,VTL11920U2,M,M,14,0.0,18-24,F,BLACK,987377.0,211980.0,40.748520,-73.988716,POINT (-73.98871585799998 40.74852036900006),b'\x12\x00\x00\x00\x01\x00\x00\x00`\xc9\xe0\x1...,105,True,NYC
5717712,205225898,11/16/2019,969.0,"TRAFFIC,UNCLASSIFIED INFRACTIO",881.0,OTHER TRAFFIC INFRACTION,VTL051101A,M,K,70,0.0,18-24,F,WHITE HISPANIC,995346.0,174216.0,40.644861,-73.960017,POINT (-73.96001731099994 40.644860566000034),b'\x12\x00\x00\x00\x01\x00\x00\x00\xac\x95r\xe...,314,True,NYC
5717713,200468223,07/31/2019,849.0,"NY STATE LAWS,UNCLASSIFIED VIO",677.0,OTHER STATE LAWS,LOC000000V,V,K,60,1.0,45-64,M,BLACK,989472.0,148971.0,40.575574,-73.981204,POINT (-73.98120362499998 40.575573630000065),b'\x12\x00\x00\x00\x01\x00\x00\x00\xdc\x05J\n\...,313,True,NYC
5717714,205012373,11/11/2019,339.0,"LARCENY,PETIT FROM OPEN AREAS,",341.0,PETIT LARCENY,PL 1552500,M,M,19,0.0,<18,M,WHITE HISPANIC,993570.0,216847.0,40.761875,-73.966358,POINT (-73.96635808399998 40.76187469000007),b'\x12\x00\x00\x00\x01\x00\x00\x00T\xc0\x93\xc...,108,True,NYC


In [ ]:
nyc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5717716 entries, 0 to 5717715
Data columns (total 23 columns):
 #   Column           Dtype  
---  ------           -----  
 0   ID               int32  
 1   Date             object 
 2   PD_CD            float64
 3   PD_DESC          object 
 4   KY_CD            float64
 5   description      object 
 6   law_code         object 
 7   LAW_CAT_CD       object 
 8   ARREST_BORO      object 
 9   ARREST_PRECINCT  int32  
 10  jurisdiction     float64
 11  age              object 
 12  sex              object 
 13  race             object 
 14  X Coordinate     float64
 15  Y Coordinate     float64
 16  Latitude         float64
 17  Longitude        float64
 18  Lon_Lat          object 
 19  geom             object 
 20  district         int64  
 21  intersects       bool   
 22  city             object 
dtypes: bool(1), float64(7), int32(2), int64(1), object(12)
memory usage: 921.5+ MB


In [ ]:
nyc_df.isnull().sum()

ID                     0
Date                   0
PD_CD                876
PD_DESC             9163
KY_CD               9750
description         9163
law_code             196
LAW_CAT_CD         23590
ARREST_BORO            8
ARREST_PRECINCT        0
jurisdiction          10
age                   17
sex                    0
race                   0
X Coordinate           0
Y Coordinate           0
Latitude               0
Longitude              0
Lon_Lat                0
geom                   0
district               0
intersects             0
city                   0
dtype: int64

In [ ]:
chicago_df.isnull().sum()

ID                           0
Case Number                  0
Date                         0
Block                        0
IUCR                         0
Primary Type                 0
Description                  0
Location Description      9120
Arrest                       0
Domestic                     0
Beat                         0
District                    47
Ward                    605084
FBI Code                     0
X Coordinate                 0
Y Coordinate                 0
Year                         0
Updated On                   0
Latitude                     0
Longitude                    0
Location                     0
geom                         0
intersects                   0
city                         0
dtype: int64

In [ ]:
chicago_df.nunique()
#nyc_df.nunique()

ID                      8177168
Case Number             8176578
Date                    3427481
Block                     63182
IUCR                        421
Primary Type                 36
Description                 569
Location Description        218
Arrest                        2
Domestic                      2
Beat                        304
District                     24
Ward                         50
FBI Code                     27
X Coordinate              79116
Y Coordinate             130268
Year                         25
Updated On                 5481
Latitude                 901972
Longitude                901374
Location                 903272
geom                     903272
intersects                    1
city                          1
dtype: int64

In [ ]:
chicago_df["Latitude"].value_counts

<bound method IndexOpsMixin.value_counts of 0          41.886018
1          41.871835
2          41.856547
3          41.655116
4          41.945232
             ...    
8177163    41.700477
8177164    41.963071
8177165    41.856231
8177166    41.866640
8177167    41.954699
Name: Latitude, Length: 8177168, dtype: float64>

In [ ]:
chicago_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8177168 entries, 0 to 8177167
Data columns (total 24 columns):
 #   Column                Dtype  
---  ------                -----  
 0   ID                    int32  
 1   Case Number           object 
 2   Date                  object 
 3   Block                 object 
 4   IUCR                  object 
 5   Primary Type          object 
 6   Description           object 
 7   Location Description  object 
 8   Arrest                bool   
 9   Domestic              bool   
 10  Beat                  int32  
 11  District              float64
 12  Ward                  float64
 13  FBI Code              object 
 14  X Coordinate          int32  
 15  Y Coordinate          int32  
 16  Year                  int32  
 17  Updated On            object 
 18  Latitude              float64
 19  Longitude             float64
 20  Location              object 
 21  geom                  object 
 22  intersects            bool   
 23  city   

In [ ]:
chicago_df.describe(include='all')

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location,geom,intersects,city
count,8.177168e+06,8177168,8177168,8177168,8177168,8177168,8177168,8168048,8177168,8177168,8.177168e+06,8.177121e+06,7.572084e+06,8177168,8.177168e+06,8.177168e+06,8.177168e+06,8177168,8.177168e+06,8.177168e+06,8177168,8177168,8177168,8177168
unique,NaN,8176578,3427481,63182,421,36,569,218,2,2,NaN,NaN,NaN,27,NaN,NaN,NaN,5481,NaN,NaN,903272,903272,1,1
top,NaN,HJ590004,01/01/2007 12:01:00 AM,001XX N STATE ST,0820,THEFT,SIMPLE,STREET,False,False,NaN,NaN,NaN,06,NaN,NaN,NaN,02/10/2018 03:50:01 PM,NaN,NaN,"(41.976290414, -87.905227221)","b""\x12\x00\x00\x00\x01\x00\x00\x007i'>\xef\xf9...",True,Chicago
freq,NaN,6,172,16602,660100,1728866,965838,2147126,6098820,6763404,NaN,NaN,NaN,1729965,NaN,NaN,NaN,2697237,NaN,NaN,14626,14626,8177168,8177168
mean,7.401516e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.182374e+03,1.127884e+01,2.274416e+01,NaN,1.164706e+06,1.885893e+06,2.010772e+03,NaN,4.184249e+01,-8.767111e+01,NaN,NaN,NaN,NaN
std,3.717804e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.029803e+02,6.953459e+00,1.385030e+01,NaN,1.618946e+04,3.139315e+04,6.897097e+00,NaN,8.633674e-02,5.892226e-02,NaN,NaN,NaN,NaN
min,6.340000e+02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.110000e+02,1.000000e+00,1.000000e+00,NaN,1.092647e+06,1.813896e+06,2.001000e+03,NaN,4.164459e+01,-8.793457e+01,NaN,NaN,NaN,NaN
25%,3.996391e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.210000e+02,6.000000e+00,1.000000e+01,NaN,1.153099e+06,1.859195e+06,2.005000e+03,NaN,4.176892e+01,-8.771321e+01,NaN,NaN,NaN,NaN
50%,7.399995e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.034000e+03,1.000000e+01,2.300000e+01,NaN,1.166222e+06,1.890879e+06,2.010000e+03,NaN,4.185622e+01,-8.766550e+01,NaN,NaN,NaN,NaN
75%,1.072816e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.731000e+03,1.700000e+01,3.400000e+01,NaN,1.176405e+06,1.909321e+06,2.016000e+03,NaN,4.190688e+01,-8.762807e+01,NaN,NaN,NaN,NaN


In [ ]:
df = spark.read.text(f"{path_data}/HI-Medium_Patterns.txt")
df.show(15, truncate=False)


+------------------------------------------------------------------------------------------------+
|value                                                                                           |
+------------------------------------------------------------------------------------------------+
|BEGIN LAUNDERING ATTEMPT - STACK                                                                |
|2022/09/01 05:14,00952,8139F54E0,0111632,8062C56E0,5331.44,US Dollar,5331.44,US Dollar,ACH,1    |
|2022/09/03 13:09,0111632,8062C56E0,008456,81363F620,5602.59,US Dollar,5602.59,US Dollar,ACH,1   |
|2022/09/01 07:40,0118693,823D5EB90,013729,801CF2E60,1400.54,US Dollar,1400.54,US Dollar,ACH,1   |
|2022/09/01 14:19,013729,801CF2E60,0123621,81A7090F0,1467.94,US Dollar,1467.94,US Dollar,ACH,1   |
|2022/09/02 12:40,0024750,81363F410,0213834,808757B00,16898.29,US Dollar,16898.29,US Dollar,ACH,1|
|2022/09/03 06:34,0213834,808757B00,000,800073EF0,17607.19,US Dollar,17607.19,US Dollar,ACH,1    |
|END LAUND

In [ ]:
df = df.withColumn("unique_id", monotonically_increasing_id())

df = df.withColumn("Suspicious_Type", 
                   when(col("value").rlike(r"BEGIN LAUNDERING ATTEMPT.*"), 
                        split(col("value"), " - ").getItem(1))
                   .otherwise(None))

window_spec = Window.orderBy("unique_id").rowsBetween(Window.unboundedPreceding, 0)

df_filled = df.withColumn("Suspicious_Type", 
                          last("Suspicious_Type", True).over(window_spec))

df_cleaned = df_filled.filter(~col("value").rlike(r"BEGIN LAUNDERING ATTEMPT|END LAUNDERING ATTEMPT"))

split_df = df_cleaned.withColumn("split_values", split(col("value"), ","))

processed_df = split_df.select(
    col("split_values").getItem(0).alias("Timestamp"),
    col("split_values").getItem(1).alias("From_ID"),
    col("split_values").getItem(2).alias("From_Account"),
    col("split_values").getItem(3).alias("To_ID"),
    col("split_values").getItem(4).alias("To_Account"),
    col("split_values").getItem(5).alias("Amount"),
    col("split_values").getItem(6).alias("Currency"),
    col("split_values").getItem(7).alias("Converted_Amount"),
    col("split_values").getItem(8).alias("Converted_Currency"),
    col("split_values").getItem(9).alias("Transaction_Type"),
    col("split_values").getItem(10).alias("Flag"),
    col("Suspicious_Type")
)

processed_df = processed_df.filter(col("Timestamp").rlike(r"^\d{4}/\d{2}/\d{2} \d{2}:\d{2}"))

processed_df.show(truncate=False)




25/04/15 18:04:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/04/15 18:04:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/04/15 18:04:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/04/15 18:04:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/04/15 18:04:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----------------+-------+------------+-------+----------+----------+---------+----------------+------------------+----------------+----+----------------------------+
|Timestamp       |From_ID|From_Account|To_ID  |To_Account|Amount    |Currency |Converted_Amount|Converted_Currency|Transaction_Type|Flag|Suspicious_Type             |
+----------------+-------+------------+-------+----------+----------+---------+----------------+------------------+----------------+----+----------------------------+
|2022/09/01 05:14|00952  |8139F54E0   |0111632|8062C56E0 |5331.44   |US Dollar|5331.44         |US Dollar         |ACH             |1   |STACK                       |
|2022/09/03 13:09|0111632|8062C56E0   |008456 |81363F620 |5602.59   |US Dollar|5602.59         |US Dollar         |ACH             |1   |STACK                       |
|2022/09/01 07:40|0118693|823D5EB90   |013729 |801CF2E60 |1400.54   |US Dollar|1400.54         |US Dollar         |ACH             |1   |STACK                       

In [ ]:
processed_df.printSchema()

root
 |-- Timestamp: string (nullable = true)
 |-- From_ID: string (nullable = true)
 |-- From_Account: string (nullable = true)
 |-- To_ID: string (nullable = true)
 |-- To_Account: string (nullable = true)
 |-- Amount: string (nullable = true)
 |-- Currency: string (nullable = true)
 |-- Converted_Amount: string (nullable = true)
 |-- Converted_Currency: string (nullable = true)
 |-- Transaction_Type: string (nullable = true)
 |-- Flag: string (nullable = true)
 |-- Suspicious_Type: string (nullable = true)



In [ ]:
def full_func_creation_table(df, db, table):
    #df.printSchema()
    null_counts = {}
    for col_name in df.columns:
        null_count = (df.withColumn("new_column", F.when((F.col(col_name).isNull()) | (F.isnan(F.col(col_name))), 1).otherwise(0)).agg(F.sum("new_column").alias("temp_sum")).collect()[0][0])

        null_counts[col_name] = null_count
    total_count = builtins.sum(null_counts.values())


    print(null_counts)
    print(total_count)
    print(f"Total count of null values: {total_count}")
    print(f"Total count of rows: {df.count()}")

    types = {"string": "String"}
    columns = [f"{column_name} {types.get(column_type, 'String')}" for column_name, column_type in df.dtypes]
    return f"create table if not exists {db}.{table} ({','.join(columns)}) ENGINE=MergeTree() order by {df.columns[0]}"


In [ ]:
null_count = df.withColumn("new_column", F.when((F.col(col_name).isNull()) | (F.isnan(F.col(col_name))), 1).otherwise(0)).agg(F.sum("new_column").alias("temp_sum")).collect()[0][0]

NameError: name 'col_name' is not defined

In [ ]:
DB = "card_data"
table = "transactions"

In [ ]:
full_func_creation_table(processed_df, DB , table)

25/02/23 17:36:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/23 17:36:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/23 17:36:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/23 17:36:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


{'Timestamp': 0, 'From_ID': 0, 'From_Account': 0, 'To_ID': 0, 'To_Account': 0, 'Amount': 0, 'Currency': 0, 'Converted_Amount': 0, 'Converted_Currency': 0, 'Transaction_Type': 0, 'Flag': 0, 'Suspicious_Type': 0}
0
Total count of null values: 0
Total count of rows: 22743


'create table if not exists card_data.transactions (Timestamp String,From_ID String,From_Account String,To_ID String,To_Account String,Amount String,Currency String,Converted_Amount String,Converted_Currency String,Transaction_Type String,Flag String,Suspicious_Type String) ENGINE=MergeTree() order by Timestamp'

In [ ]:
null_count = processed_df.withColumn("new_column", F.when(F.col(col_name).isNull()) | (F.isnan(F.col(col_name))), 1).otherwise(0)

In [ ]:
processed_df.dtypes

[('Timestamp', 'string'),
 ('From_ID', 'string'),
 ('From_Account', 'string'),
 ('To_ID', 'string'),
 ('To_Account', 'string'),
 ('Amount', 'string'),
 ('Currency', 'string'),
 ('Converted_Amount', 'string'),
 ('Converted_Currency', 'string'),
 ('Transaction_Type', 'string'),
 ('Flag', 'string'),
 ('Suspicious_Type', 'string')]

In [ ]:
types = {"string": "String"}

columns = [f"{column_name} {types.get(column_type, 'String')}" for column_name, column_type in processed_df.dtypes]
print(columns)

['Timestamp String', 'From_ID String', 'From_Account String', 'To_ID String', 'To_Account String', 'Amount String', 'Currency String', 'Converted_Amount String', 'Converted_Currency String', 'Transaction_Type String', 'Flag String', 'Suspicious_Type String']


In [ ]:
DB = "card_data"
table = "transactions"

In [ ]:
print(f"create table if not exists {DB}.{table} ({','.join(columns)}) ENGINE=MergeTree() order by {processed_df.columns[0]}")  
print("Created")

create table if not exists card_data.transactions (Timestamp String,From_ID String,From_Account String,To_ID String,To_Account String,Amount String,Currency String,Converted_Amount String,Converted_Currency String,Transaction_Type String,Flag String,Suspicious_Type String) ENGINE=MergeTree() order by Timestamp
Created


In [ ]:
def ddl_func(df, db, table):
    types = {"string": "String"}
    columns = [f"{column_name} {types.get(column_type, 'String')}" for column_name, column_type in df.dtypes]
    return f"create table if not exists {db}.{table} ({','.join(columns)}) ENGINE=MergeTree() order by {df.columns[0]}"

In [ ]:
ddl_func(processed_df, DB, table)

'create table if not exists card_data.transactions (Timestamp String,From_ID String,From_Account String,To_ID String,To_Account String,Amount String,Currency String,Converted_Amount String,Converted_Currency String,Transaction_Type String,Flag String,Suspicious_Type String) ENGINE=MergeTree() order by Timestamp'

In [ ]:
','.join(columns)

'Timestamp String,From_ID String,From_Account String,To_ID String,To_Account String,Amount String,Currency String,Converted_Amount String,Converted_Currency String,Transaction_Type String,Flag String,Suspicious_Type String'

In [ ]:
types.get('string', 'String')

'String'

In [ ]:

grouped_by_currency = processed_df.groupBy("Currency").agg(
        F.count("*").alias("Transaction_Count"),
        F.sum("Amount").alias("Total_Amount")
    )
grouped_by_currency.show()

+-----------------+-----------------+--------------------+
|         Currency|Transaction_Count|        Total_Amount|
+-----------------+-----------------+--------------------+
|              Yen|              819|1.242312435171999...|
|         UK Pound|             1262|      7.3326693163E8|
|Australian Dollar|              341|4.1358344824000007E8|
|      Saudi Riyal|                8|  372465.44000000006|
|     Mexican Peso|                4|           953579.44|
|           Shekel|              114| 2.705172844310001E9|
|             Yuan|             1882|2.191043557284299...|
|  Canadian Dollar|              141|2.5041100439000005E8|
|             Euro|             6792|1.305416974135001...|
|            Rupee|              552|      5.0259031931E9|
|      Swiss Franc|                4|            18735.84|
|        US Dollar|             9714|2.669610893167986E10|
|      Brazil Real|               20|      3.1150056855E8|
|          Bitcoin|                3|            3.62589

In [ ]:
grouped_by_transaction_type = processed_df.groupBy("Transaction_Type").agg(
        F.count("*").alias("Transaction_Count"),
        F.sum("Amount").alias("Total_Amount")
    )
grouped_by_transaction_type.show()

+----------------+-----------------+--------------------+
|Transaction_Type|Transaction_Count|        Total_Amount|
+----------------+-----------------+--------------------+
|             ACH|            22740|1.667395935097328...|
|         Bitcoin|                3|            3.625899|
+----------------+-----------------+--------------------+



In [ ]:
grouped_by_flag = processed_df.groupBy("Flag").agg(
        F.count("*").alias("Transaction_Count"),
        F.sum("Amount").alias("Total_Amount")
    )
grouped_by_flag.show()

+----+-----------------+--------------------+
|Flag|Transaction_Count|        Total_Amount|
+----+-----------------+--------------------+
|   1|            22743|1.667395935100954...|
+----+-----------------+--------------------+



In [ ]:
grouped_by_sender = processed_df.groupBy("From_ID").agg(
        F.count("*").alias("Transaction_Count"),
        F.sum("Amount").alias("Total_Amount")
    )
grouped_by_sender.show()

+--------+-----------------+------------------+
| From_ID|Transaction_Count|      Total_Amount|
+--------+-----------------+------------------+
| 0212474|               19| 777334.5499999999|
| 0253580|                4|         1077850.3|
| 0125414|                4|          45690.69|
| 0173659|                2|105983.34999999999|
| 0031953|                3|          32217.49|
|02127232|                1|           9976.75|
| 0111606|               20|        8239124.81|
| 0014993|                9| 655120.6699999999|
| 0224393|                5|         121904.21|
| 0036435|                3|30379.440000000002|
| 0172156|               15|        6227672.95|
|  025699|               33| 5735640.220000001|
|  015240|               22|     4.767193925E7|
| 0254896|                2|          14922.56|
| 0152922|                3|          32692.75|
| 0078860|                2|           8395.76|
| 0055436|                5|         997488.92|
| 0054349|                1|         979

In [ ]:
avg_amount_by_currency = processed_df.groupBy("Currency").agg(F.avg("Amount").alias("Average_Amount"))
avg_amount_by_currency.show()

+-----------------+--------------------+
|         Currency|      Average_Amount|
+-----------------+--------------------+
|              Yen|1.5168650002100117E9|
|         UK Pound|   581035.6035103011|
|Australian Dollar|  1212854.6869208212|
|      Saudi Riyal|   46558.18000000001|
|     Mexican Peso|           238394.86|
|           Shekel|2.3729586353596497E7|
|             Yuan|1.1642101792158872E8|
|  Canadian Dollar|  1775964.5701418442|
|             Euro|  1921992.0113883999|
|            Rupee|   9104897.088949276|
|      Swiss Franc|             4683.96|
|        US Dollar|   2748209.690310877|
|      Brazil Real|     1.55750284275E7|
|          Bitcoin|            1.208633|
|            Ruble| 1.442388985767527E8|
+-----------------+--------------------+



In [ ]:
max_amount_by_suspicious_type = processed_df.groupBy("Suspicious_Type").agg(F.max("Amount").alias("Max_Amount"))
max_amount_by_suspicious_type.show()

25/02/03 19:24:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/03 19:24:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/03 19:24:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/03 19:24:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------------------+-------------+
|     Suspicious_Type|   Max_Amount|
+--------------------+-------------+
|           BIPARTITE|    999845.79|
| CYCLE:  Max 10 hops|      9951.36|
| CYCLE:  Max 11 hops|     96739.20|
| CYCLE:  Max 12 hops|      9872.20|
| CYCLE:  Max 13 hops|      9971.68|
|  CYCLE:  Max 2 hops|      9934.24|
|  CYCLE:  Max 3 hops|    973986.67|
|  CYCLE:  Max 4 hops|    990715.61|
|  CYCLE:  Max 5 hops|      9783.97|
|  CYCLE:  Max 6 hops|9986928829.16|
|  CYCLE:  Max 7 hops|     98728.19|
|  CYCLE:  Max 8 hops|      9811.06|
|  CYCLE:  Max 9 hops|      9586.18|
|FAN-IN:  Max 1-de...|      9762.64|
|FAN-IN:  Max 10-d...|     99520.51|
|FAN-IN:  Max 11-d...|      9919.49|
|FAN-IN:  Max 12-d...|      9976.75|
|FAN-IN:  Max 13-d...|      9965.87|
|FAN-IN:  Max 14-d...|      9978.47|
|FAN-IN:  Max 15-d...|     99560.78|
+--------------------+-------------+
only showing top 20 rows



In [ ]:
total_transactions_by_flag = processed_df.groupBy("Flag").agg(F.count("*").alias("Total_Transactions"))
total_transactions_by_flag.show()

+----+------------------+
|Flag|Total_Transactions|
+----+------------------+
|   1|             22743|
+----+------------------+



In [ ]:
window_spec_rank = Window.partitionBy("Currency").orderBy(F.desc("Amount"))
processed_df = processed_df.withColumn("Rank", F.rank().over(window_spec_rank))
processed_df.show()

25/02/03 19:24:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/03 19:24:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/03 19:24:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/03 19:24:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----------------+--------+------------+--------+----------+-------+-----------------+----------------+------------------+----------------+----+--------------------+----+
|       Timestamp| From_ID|From_Account|   To_ID|To_Account| Amount|         Currency|Converted_Amount|Converted_Currency|Transaction_Type|Flag|     Suspicious_Type|Rank|
+----------------+--------+------------+--------+----------+-------+-----------------+----------------+------------------+----------------+----+--------------------+----+
|2022/09/18 15:01|02127147|   82F31CAE0|02127147| 83000CAC0|9685.09|Australian Dollar|         9685.09| Australian Dollar|             ACH|   1|      SCATTER-GATHER|   1|
|2022/09/14 12:31|    0111|   800065010|     029| 8000C7720|9521.79|Australian Dollar|         9521.79| Australian Dollar|             ACH|   1|FAN-IN:  Max 14-d...|   2|
|2022/09/06 10:25|    0110|   82E6C08C0|00126614| 82E6C0F40|9471.55|Australian Dollar|         9471.55| Australian Dollar|             ACH|   1|G

In [ ]:
window_spec_max = Window.partitionBy("Suspicious_Type").orderBy(F.desc("Amount"))
processed_df = processed_df.withColumn("Max_Amount_In_Suspicious_Type", F.max("Amount").over(window_spec_max))
processed_df.show()

25/02/03 19:24:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/03 19:24:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/03 19:24:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/03 19:24:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----------------+-------+------------+--------+----------+---------+---------+----------------+------------------+----------------+----+---------------+----+-----------------------------+
|       Timestamp|From_ID|From_Account|   To_ID|To_Account|   Amount| Currency|Converted_Amount|Converted_Currency|Transaction_Type|Flag|Suspicious_Type|Rank|Max_Amount_In_Suspicious_Type|
+----------------+-------+------------+--------+----------+---------+---------+----------------+------------------+----------------+----+---------------+----+-----------------------------+
|2022/09/16 15:13|   0113|   8000D8040|     005| 80EE7B9A0|999845.79|      Yen|       999845.79|               Yen|             ACH|   1|      BIPARTITE|   1|                    999845.79|
|2022/09/04 12:03|    011|   8002CA100|   00544| 8003AB050|  9996.32|US Dollar|         9996.32|         US Dollar|             ACH|   1|      BIPARTITE|   1|                    999845.79|
|2022/09/09 04:32| 022819|   8035F8070|   00544| 8018C6

In [ ]:

processed_df = processed_df.withColumn("Amount", F.col("Amount").cast(DoubleType()))

def amount_category(amount):
    if amount < 1000:
        return "Small"
    elif 1000 <= amount < 10000:
        return "Medium"
    else:
        return "Large"

udf_amount_category = F.udf(amount_category, StringType())

processed_df = processed_df.withColumn("Amount_Category", udf_amount_category(F.col("Amount")))

processed_df.show()

25/02/03 19:24:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/03 19:24:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/03 19:24:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/03 19:24:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----------------+-------+------------+--------+----------+---------+---------+----------------+------------------+----------------+----+---------------+----+-----------------------------+---------------+
|       Timestamp|From_ID|From_Account|   To_ID|To_Account|   Amount| Currency|Converted_Amount|Converted_Currency|Transaction_Type|Flag|Suspicious_Type|Rank|Max_Amount_In_Suspicious_Type|Amount_Category|
+----------------+-------+------------+--------+----------+---------+---------+----------------+------------------+----------------+----+---------------+----+-----------------------------+---------------+
|2022/09/16 15:13|   0113|   8000D8040|     005| 80EE7B9A0|999845.79|      Yen|       999845.79|               Yen|             ACH|   1|      BIPARTITE|   1|                    999845.79|          Large|
|2022/09/04 12:03|    011|   8002CA100|   00544| 8003AB050|  9996.32|US Dollar|         9996.32|         US Dollar|             ACH|   1|      BIPARTITE|   1|                    99

In [ ]:
def is_large_transaction(amount):
    return amount > 10000

udf_is_large_transaction = F.udf(is_large_transaction, BooleanType())
processed_df = processed_df.withColumn("Is_Large_Transaction", udf_is_large_transaction(F.col("Amount")))
processed_df.show()

25/02/03 19:24:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/03 19:24:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/03 19:24:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/03 19:24:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----------------+-------+------------+--------+----------+---------+---------+----------------+------------------+----------------+----+---------------+----+-----------------------------+---------------+--------------------+
|       Timestamp|From_ID|From_Account|   To_ID|To_Account|   Amount| Currency|Converted_Amount|Converted_Currency|Transaction_Type|Flag|Suspicious_Type|Rank|Max_Amount_In_Suspicious_Type|Amount_Category|Is_Large_Transaction|
+----------------+-------+------------+--------+----------+---------+---------+----------------+------------------+----------------+----+---------------+----+-----------------------------+---------------+--------------------+
|2022/09/16 15:13|   0113|   8000D8040|     005| 80EE7B9A0|999845.79|      Yen|       999845.79|               Yen|             ACH|   1|      BIPARTITE|   1|                    999845.79|          Large|                true|
|2022/09/04 12:03|    011|   8002CA100|   00544| 8003AB050|  9996.32|US Dollar|         9996.32|

In [ ]:
processed_df = processed_df.withColumn("Date", F.to_date(F.col("Timestamp")))
processed_df = processed_df.withColumn("Time", F.date_format(F.col("Timestamp"), "HH:mm:ss"))
processed_df.show()

25/02/03 19:25:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/03 19:25:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/03 19:25:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/03 19:25:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----------------+-------+------------+--------+----------+---------+---------+----------------+------------------+----------------+----+---------------+----+-----------------------------+---------------+--------------------+----+----+
|       Timestamp|From_ID|From_Account|   To_ID|To_Account|   Amount| Currency|Converted_Amount|Converted_Currency|Transaction_Type|Flag|Suspicious_Type|Rank|Max_Amount_In_Suspicious_Type|Amount_Category|Is_Large_Transaction|Date|Time|
+----------------+-------+------------+--------+----------+---------+---------+----------------+------------------+----------------+----+---------------+----+-----------------------------+---------------+--------------------+----+----+
|2022/09/16 15:13|   0113|   8000D8040|     005| 80EE7B9A0|999845.79|      Yen|       999845.79|               Yen|             ACH|   1|      BIPARTITE|   1|                    999845.79|          Large|                true|NULL|NULL|
|2022/09/04 12:03|    011|   8002CA100|   00544| 8003AB0

In [ ]:
processed_df = processed_df.withColumn("Year", F.year(F.col("Timestamp")))
processed_df = processed_df.withColumn("Month", F.month(F.col("Timestamp")))
processed_df = processed_df.withColumn("Day", F.dayofmonth(F.col("Timestamp")))

processed_df.show()


25/02/03 19:25:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/03 19:25:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/03 19:25:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/03 19:25:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----------------+-------+------------+--------+----------+---------+---------+----------------+------------------+----------------+----+---------------+----+-----------------------------+---------------+--------------------+----+----+----+-----+----+
|       Timestamp|From_ID|From_Account|   To_ID|To_Account|   Amount| Currency|Converted_Amount|Converted_Currency|Transaction_Type|Flag|Suspicious_Type|Rank|Max_Amount_In_Suspicious_Type|Amount_Category|Is_Large_Transaction|Date|Time|Year|Month| Day|
+----------------+-------+------------+--------+----------+---------+---------+----------------+------------------+----------------+----+---------------+----+-----------------------------+---------------+--------------------+----+----+----+-----+----+
|2022/09/16 15:13|   0113|   8000D8040|     005| 80EE7B9A0|999845.79|      Yen|       999845.79|               Yen|             ACH|   1|      BIPARTITE|   1|                    999845.79|          Large|                true|NULL|NULL|NULL| NUL

In [ ]:
processed_df = processed_df.withColumn("Timestamp", F.to_timestamp(F.col("Timestamp"), "yyyy/MM/dd HH:mm"))
processed_df.printSchema()

root
 |-- Timestamp: timestamp (nullable = true)
 |-- From_ID: string (nullable = true)
 |-- From_Account: string (nullable = true)
 |-- To_ID: string (nullable = true)
 |-- To_Account: string (nullable = true)
 |-- Amount: double (nullable = true)
 |-- Currency: string (nullable = true)
 |-- Converted_Amount: string (nullable = true)
 |-- Converted_Currency: string (nullable = true)
 |-- Transaction_Type: string (nullable = true)
 |-- Flag: string (nullable = true)
 |-- Suspicious_Type: string (nullable = true)
 |-- Rank: integer (nullable = false)
 |-- Max_Amount_In_Suspicious_Type: string (nullable = true)
 |-- Amount_Category: string (nullable = true)
 |-- Is_Large_Transaction: boolean (nullable = true)
 |-- Date: date (nullable = true)
 |-- Time: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Day: integer (nullable = true)



In [ ]:
processed_df = processed_df.withColumn("Date", F.to_date(F.col("Timestamp")))

processed_df = processed_df.withColumn("Time", F.date_format(F.col("Timestamp"), "HH:mm:ss"))

processed_df = processed_df.withColumn("Year", F.year(F.col("Timestamp")))
processed_df = processed_df.withColumn("Month", F.month(F.col("Timestamp")))
processed_df = processed_df.withColumn("Day", F.dayofmonth(F.col("Timestamp")))

In [ ]:
processed_df.show()

25/02/03 19:25:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/03 19:25:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/03 19:25:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/03 19:25:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------------------+-------+------------+--------+----------+---------+---------+----------------+------------------+----------------+----+---------------+----+-----------------------------+---------------+--------------------+----------+--------+----+-----+---+
|          Timestamp|From_ID|From_Account|   To_ID|To_Account|   Amount| Currency|Converted_Amount|Converted_Currency|Transaction_Type|Flag|Suspicious_Type|Rank|Max_Amount_In_Suspicious_Type|Amount_Category|Is_Large_Transaction|      Date|    Time|Year|Month|Day|
+-------------------+-------+------------+--------+----------+---------+---------+----------------+------------------+----------------+----+---------------+----+-----------------------------+---------------+--------------------+----------+--------+----+-----+---+
|2022-09-16 15:13:00|   0113|   8000D8040|     005| 80EE7B9A0|999845.79|      Yen|       999845.79|               Yen|             ACH|   1|      BIPARTITE|   1|                    999845.79|          Large| 

In [ ]:
from pyspark.sql.functions import split, col


split_df = df.withColumn("split_values", split(col("value"), ","))

processed_df = split_df.select(
    col("split_values").getItem(0).alias("Timestamp"),
    col("split_values").getItem(1).alias("From_ID"),
    col("split_values").getItem(2).alias("From_Account"),
    col("split_values").getItem(3).alias("To_ID"),
    col("split_values").getItem(4).alias("To_Account"),
    col("split_values").getItem(5).alias("Amount"),
    col("split_values").getItem(6).alias("Currency"),
    col("split_values").getItem(7).alias("Converted_Amount"),
    col("split_values").getItem(8).alias("Converted_Currency"),
    col("split_values").getItem(9).alias("Transaction_Type"),
    col("split_values").getItem(10).alias("Flag")
)
processed_df = processed_df.filter(col("Timestamp").rlike(r"\d{4}/\d{2}/\d{2} \d{2}:\d{2}"))

processed_df.show(15, truncate=False)

+----------------+-------+------------+-------+----------+---------+---------+----------------+------------------+----------------+----+
|Timestamp       |From_ID|From_Account|To_ID  |To_Account|Amount   |Currency |Converted_Amount|Converted_Currency|Transaction_Type|Flag|
+----------------+-------+------------+-------+----------+---------+---------+----------------+------------------+----------------+----+
|2022/09/01 05:14|00952  |8139F54E0   |0111632|8062C56E0 |5331.44  |US Dollar|5331.44         |US Dollar         |ACH             |1   |
|2022/09/03 13:09|0111632|8062C56E0   |008456 |81363F620 |5602.59  |US Dollar|5602.59         |US Dollar         |ACH             |1   |
|2022/09/01 07:40|0118693|823D5EB90   |013729 |801CF2E60 |1400.54  |US Dollar|1400.54         |US Dollar         |ACH             |1   |
|2022/09/01 14:19|013729 |801CF2E60   |0123621|81A7090F0 |1467.94  |US Dollar|1467.94         |US Dollar         |ACH             |1   |
|2022/09/02 12:40|0024750|81363F410   |02

In [ ]:
spark = SparkSession.builder.appName("TransactionAnalysis").getOrCreate()

grouped_df = processed_df.groupBy("Currency").agg(
    sum("Amount").alias("Total_Amount"),
    avg("Amount").alias("Average_Amount"),
    count("Amount").alias("Transaction_Count")
)


grouped_df.show()

25/02/01 22:37:10 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+-----------------+--------------------+--------------------+-----------------+
|         Currency|        Total_Amount|      Average_Amount|Transaction_Count|
+-----------------+--------------------+--------------------+-----------------+
|              Yen|1.242312435171999...|1.5168650002100117E9|              819|
|         UK Pound|      7.3326693163E8|   581035.6035103011|             1262|
|Australian Dollar|4.1358344824000007E8|  1212854.6869208212|              341|
|      Saudi Riyal|  372465.44000000006|   46558.18000000001|                8|
|     Mexican Peso|           953579.44|           238394.86|                4|
|           Shekel| 2.705172844310001E9|2.3729586353596497E7|              114|
|             Yuan|2.191043557284299...|1.1642101792158872E8|             1882|
|  Canadian Dollar|2.5041100439000005E8|  1775964.5701418442|              141|
|             Euro|1.305416974135001...|  1921992.0113883999|             6792|
|            Rupee|      5.0259031931E9|

In [ ]:

window_spec = Window.partitionBy("Currency").orderBy("Amount")

df_with_rank = processed_df.withColumn("row_number", row_number().over(window_spec)) \
                           .withColumn("rank", rank().over(window_spec)) \
                           .withColumn("dense_rank", dense_rank().over(window_spec))

df_with_rank.show(250)

+----------------+--------+------------+--------+----------+------------+-----------------+----------------+------------------+----------------+----+----------+----+----------+
|       Timestamp| From_ID|From_Account|   To_ID|To_Account|      Amount|         Currency|Converted_Amount|Converted_Currency|Transaction_Type|Flag|row_number|rank|dense_rank|
+----------------+--------+------------+--------+----------+------------+-----------------+----------------+------------------+----------------+----+----------+----+----------+
|2022/09/04 02:04|01136230|   833638190|01125022| 833638230|     1005.71|Australian Dollar|         1005.71| Australian Dollar|             ACH|   1|         1|   1|         1|
|2022/09/08 02:45|02126306|   83000CE40|01125491| 83000DC40|    10089.17|Australian Dollar|        10089.17| Australian Dollar|             ACH|   1|         2|   2|         2|
|2022/09/12 03:18| 0239197|   83000D0D0|02129461| 83000DC90|    10113.08|Australian Dollar|        10113.08| Austra

In [ ]:


def calculate_tax(amount):
    try:
        return float(amount) * 0.2
    except (TypeError, ValueError):
        return 0.0 

calculate_tax_udf = udf(calculate_tax, DoubleType())

df_with_tax = processed_df.withColumn("Tax", calculate_tax_udf(col("Amount")))

df_with_tax.show()

+----------------+-------+------------+-------+----------+----------+---------+----------------+------------------+----------------+----+------------------+
|       Timestamp|From_ID|From_Account|  To_ID|To_Account|    Amount| Currency|Converted_Amount|Converted_Currency|Transaction_Type|Flag|               Tax|
+----------------+-------+------------+-------+----------+----------+---------+----------------+------------------+----------------+----+------------------+
|2022/09/01 05:14|  00952|   8139F54E0|0111632| 8062C56E0|   5331.44|US Dollar|         5331.44|         US Dollar|             ACH|   1|          1066.288|
|2022/09/03 13:09|0111632|   8062C56E0| 008456| 81363F620|   5602.59|US Dollar|         5602.59|         US Dollar|             ACH|   1|          1120.518|
|2022/09/01 07:40|0118693|   823D5EB90| 013729| 801CF2E60|   1400.54|US Dollar|         1400.54|         US Dollar|             ACH|   1|           280.108|
|2022/09/01 14:19| 013729|   801CF2E60|0123621| 81A7090F0|